In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
from scipy.sparse.linalg import spsolve
from lmfit.models import Model, LorentzianModel, QuadraticModel, PolynomialModel, LinearModel, ConstantModel

In [2]:
#%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)

In [3]:
def baseline(y, lam=10**9, p=0.01, niter=10):
    """
    For more information see: http://zanran_storage.s3.amazonaws.com/www.science.uva.nl/ContentPages/443199618.pdf
    lam: is a parameter for the smoothness and values between 10^2 and 10^9 are good for poitiv peaks
    p: is a paramter for aysymmetry and values between 0.001 and 0.1 are a good choice 
    """
    l = len(y)
    D = sparse.csc_matrix(np.diff(np.eye(l), 2))
    w = np.ones(l)
    for i in range(niter):
        W = sparse.spdiags(w, 0, l, l)
        Z = W + lam * D.dot(D.T)
        z = sparse.linalg.spsolve(Z, w*y)
        w = p * (y > z) + (1-p) * (y < z)
    return z

# Beispiel

In [4]:
y = 1/np.exp(0.003*np.arange(0,1000)) + np.sin(0.03*np.arange(0,1000))

In [5]:
plt.plot(np.arange(0,1000),y)
plt.show()

In [6]:
plt.plot(np.arange(0,1,0.001),baseline(y,p=0.001,lam=1.5317*10**6), '-.b', label="Baseline")
plt.plot(np.arange(0,1,0.001),y-baseline(y,p=0.001,lam=1.5317*10**6)-1,'-r', label="Data without underground")
plt.plot(np.arange(0,1,0.001),y, '--g', label="Data with underground")
plt.legend(loc="best")
plt.show()

# Datenanalyse

In [7]:
data_messung_1 = pd.read_csv("Messung_1.dat", delim_whitespace=True)

In [8]:
Prfl = -(data_messung_1.iloc[8].Amplitude/(np.exp(0.5)-1))
delta_Prfl = (0.1/(np.exp(0.5)-1))


In [9]:
data_messung_1["err"] = np.where(data_messung_1.Amplitude < 1, 0.01, 0.1)

In [10]:
data_messung_1["absorptionskoeffizient"] = 10/5*np.log(1-data_messung_1.Amplitude/Prfl)
data_messung_1["absorp_err"] = (((2*data_messung_1.err)/(data_messung_1.Amplitude-Prfl))**2 + ((2*delta_Prfl*data_messung_1.Amplitude)/(Prfl*data_messung_1.Amplitude-data_messung_1.Amplitude**2))**2)**0.5

In [11]:
data_messung_1.sort_values("Amplitude")[::-1]

,Nummer,J,K,nu_theo,nu_gemessen,Amplitude,err,absorptionskoeffizient,absorp_err
8,31,3,3,23.870,23.877,20.000,0.10,1.000000,0.007230
3,36,6,6,25.056,25.061,19.700,0.10,0.988161,0.007273
7,32,4,4,24.139,24.140,15.100,0.10,0.797263,0.008001
17,22,4,3,22.687,22.695,14.700,0.10,0.779769,0.008071
5,34,5,5,24.533,24.538,14.500,0.10,0.770964,0.008107
1,38,7,7,25.715,25.714,11.100,0.10,0.615029,0.008764
18,21,5,4,22.654,22.660,10.200,0.10,0.571632,0.008957
9,30,2,2,23.723,23.730,9.700,0.10,0.547110,0.009067
10,29,1,1,23.694,23.700,9.150,0.10,0.519784,0.009192
14,25,7,6,22.927,22.929,8.600,0.10,0.492079,0.009320


In [933]:
print(data_messung_1.sort_values("Nummer")[::1].to_latex(index=False))

\begin{tabular}{rrrrrrrrr}
\toprule
 Nummer &   J &   K &  nu\_theo &  nu\_gemessen &  Amplitude &   err &  absorptionskoeffizient &  absorp\_err \\
\midrule
      1 &   7 &   3 &   18.058 &       18.028 &      0.220 &  0.01 &                0.014221 &    0.009950 \\
      2 &  10 &   7 &   18.282 &       18.294 &      0.100 &  0.01 &                0.006477 &    0.009989 \\
      3 &   6 &   1 &   18.382 &       18.395 &      0.090 &  0.01 &                0.005830 &    0.009992 \\
      4 &   9 &   6 &   18.495 &       18.509 &      0.410 &  0.01 &                0.026422 &    0.009890 \\
      5 &   8 &   5 &   18.819 &       18.819 &      0.450 &  0.01 &                0.028981 &    0.009877 \\
      6 &   6 &   2 &   18.879 &       18.891 &      0.310 &  0.01 &                0.020010 &    0.009921 \\
      7 &   7 &   4 &   19.216 &       19.225 &      0.210 &  0.01 &                0.013577 &    0.009953 \\
      8 &   6 &   3 &   19.735 &       19.764 &      1.400 &  0.10 &    

In [924]:
plt.bar(data_messung_1.nu_gemessen,data_messung_1.absorptionskoeffizient,0.04,
        yerr=data_messung_1.absorp_err,
        color="g",
        error_kw={'ecolor': 'r'})
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.ylabel(r'$\alpha$ in dB/m', fontsize=24)
plt.xlabel("Frequency/GHz", fontsize=24)
plt.ylim([0,1.1])
plt.show()

# Hyperfinestrucktur

In [13]:
peak_4_4 = np.genfromtxt("Frederik_Henrik/Messung_2/H_F_4_4.tab", skip_header=1).T
peak_4_4_kalib_1 = np.genfromtxt("Frederik_Henrik/Messung_2/H_F_4_4_24_130.tab", skip_header=1).T
peak_4_4_kalib_2 = np.genfromtxt("Frederik_Henrik/Messung_2/H_F_4_4_24_145.tab", skip_header=1).T
peak_4_4_kalib_3 = np.genfromtxt("Frederik_Henrik/Messung_2/H_F_4_4_24_149.tab", skip_header=1).T

### Untergrund

In [14]:
plt.plot(peak_4_4[0],peak_4_4[1], ' .')
plt.show()

In [15]:
submask_untergrund_hf_4_4_upper = np.where(peak_4_4[0] > 0.03, True, False)
submask_untergrund_hf_4_4_lower = np.where(peak_4_4[0] < 0.02, True,False)
mask_untergrund_hf_4_4 = np.where(np.logical_or(submask_untergrund_hf_4_4_upper,
                                                 submask_untergrund_hf_4_4_lower))

In [16]:
mod_underground_hf_4_4 = PolynomialModel(4)

In [17]:
pars_underground_hf_4_4 = mod_underground_hf_4_4.guess(peak_4_4[1][mask_untergrund_hf_4_4],
                                                         x=peak_4_4[0][mask_untergrund_hf_4_4])

In [18]:
out_underground_hf_4_4 = mod_underground_hf_4_4.fit(peak_4_4[1][mask_untergrund_hf_4_4],
                                                      params=pars_underground_hf_4_4,
                                                      x=peak_4_4[0][mask_untergrund_hf_4_4],
                                                      weights=1/0.0001)

In [19]:
print(out_underground_hf_4_4.fit_report(show_correl=False))

[[Model]]
    Model(polynomial)
[[Fit Statistics]]
    # function evals   = 8
    # data points      = 1645
    # variables        = 5
    chi-square         = 514.289
    reduced chi-square = 0.314
[[Variables]]
    c0:   0.00052059 +/- 6.32e-06 (1.21%) (init= 0.0005205947)
    c1:   0.24040414 +/- 0.001833 (0.76%) (init= 0.2404041)
    c2:  -32.9505331 +/- 0.157074 (0.48%) (init=-32.95053)
    c3:   1110.99435 +/- 4.758532 (0.43%) (init= 1110.994)
    c4:  -10777.6796 +/- 46.41823 (0.43%) (init=-10777.68)



In [20]:
out_underground_hf_4_4.plot()
plt.show()

In [21]:
plt.plot(peak_4_4[0],peak_4_4[1], ' .')
plt.plot(np.arange(0,0.052,0.001),out_underground_hf_4_4.eval(x=np.arange(0,0.052,0.001)))
plt.show()

In [22]:
untergrund_hf_4_4 = out_underground_hf_4_4.eval(x=peak_4_4[0])

In [23]:
plt.plot(peak_4_4[0],peak_4_4[1]-untergrund_hf_4_4, ' .')
plt.show()

### Kalibration

#### Peak 1

In [24]:
kalib_hf_4_4_peak_1_new = peak_4_4_kalib_1[1]-untergrund_hf_4_4

In [25]:
plt.plot(peak_4_4_kalib_1[0],kalib_hf_4_4_peak_1_new, ' .')
plt.show()

In [26]:
mask_kalib_hf_4_4_peak_1 = np.where(peak_4_4_kalib_1[0][np.where(peak_4_4_kalib_1[0] < 0.01)[0]] > 0.0045)[0]

In [27]:
mod_kalib_hf_4_4_peak_1 = LorentzianModel()
mod_kalib_const_hf_4_4_peak_1 = ConstantModel()

In [28]:
pars_kalib_hf_4_4_peak_1 = mod_kalib_hf_4_4_peak_1.guess(kalib_hf_4_4_peak_1_new[mask_kalib_hf_4_4_peak_1],
                                                         x=peak_4_4_kalib_1[0][mask_kalib_hf_4_4_peak_1])
pars_kalib_hf_4_4_peak_1["sigma"].set(0.001)
pars_kalib_hf_4_4_peak_1["center"].set(0.0081)
pars_kalib_hf_4_4_peak_1["amplitude"].set(-0.001)

In [29]:
out_kalib_hf_4_4_peak_1 = mod_kalib_hf_4_4_peak_1.fit(kalib_hf_4_4_peak_1_new[mask_kalib_hf_4_4_peak_1],
                                                      params=pars_kalib_hf_4_4_peak_1,
                                                      x=peak_4_4_kalib_1[0][mask_kalib_hf_4_4_peak_1],
                                                      weights=1/(kalib_hf_4_4_peak_1_new[mask_kalib_hf_4_4_peak_1]*0.02))

In [30]:
print(out_kalib_hf_4_4_peak_1.fit_report(show_correl=False))

[[Model]]
    Model(lorentzian)
[[Fit Statistics]]
    # function evals   = 35
    # data points      = 219
    # variables        = 3
    chi-square         = 161.423
    reduced chi-square = 0.747
[[Variables]]
    sigma:       0.00685879 +/- 9.60e-05 (1.40%) (init= 0.001)
    fwhm:        0.01371758 +/- 0.000192 (1.40%)  == '2.0000000*sigma'
    center:      0.00784729 +/- 2.38e-05 (0.30%) (init= 0.0081)
    amplitude:  -0.00020719 +/- 2.66e-06 (1.28%) (init=-0.001)



In [31]:
out_kalib_hf_4_4_peak_1.plot()
plt.show()

In [32]:
plt.plot(peak_4_4_kalib_1[0],kalib_hf_4_4_peak_1_new)
plt.plot(np.arange(0,0.052,0.00001),out_kalib_hf_4_4_peak_1.eval(x=np.arange(0,0.052,0.00001)))
plt.show()

#### Peak 2

In [33]:
kalib_hf_4_4_peak_2_new = peak_4_4_kalib_2[1]-untergrund_hf_4_4

In [34]:
plt.plot(peak_4_4_kalib_2[0],kalib_hf_4_4_peak_2_new, ' .')
plt.show()

In [35]:
mask_kalib_hf_4_4_peak_2 = np.where(peak_4_4_kalib_2[0][np.where(peak_4_4_kalib_2[0] < 0.043)[0]] > 0.0324)[0]

In [36]:
mod_kalib_hf_4_4_peak_2 = LorentzianModel()
mod_kalib_const_hf_4_4_peak_2 = ConstantModel()

In [37]:
pars_kalib_hf_4_4_peak_2 = mod_kalib_hf_4_4_peak_2.guess(kalib_hf_4_4_peak_2_new[mask_kalib_hf_4_4_peak_2],
                                                         x=peak_4_4_kalib_2[0][mask_kalib_hf_4_4_peak_2])
pars_kalib_hf_4_4_peak_2["sigma"].set(0.001)
pars_kalib_hf_4_4_peak_2["center"].set(0.0081)
pars_kalib_hf_4_4_peak_2["amplitude"].set(-0.001)

In [38]:
out_kalib_hf_4_4_peak_2 = mod_kalib_hf_4_4_peak_2.fit(kalib_hf_4_4_peak_2_new[mask_kalib_hf_4_4_peak_2],
                                                      params=pars_kalib_hf_4_4_peak_2,
                                                      x=peak_4_4_kalib_2[0][mask_kalib_hf_4_4_peak_2],
                                                      weights=1/(kalib_hf_4_4_peak_2_new[mask_kalib_hf_4_4_peak_2]*0.02))

In [39]:
print(out_kalib_hf_4_4_peak_2.fit_report(show_correl=False))

[[Model]]
    Model(lorentzian)
[[Fit Statistics]]
    # function evals   = 131
    # data points      = 423
    # variables        = 3
    chi-square         = 4348.240
    reduced chi-square = 10.353
[[Variables]]
    sigma:       0.00461866 +/- 3.81e-05 (0.83%) (init= 0.001)
    fwhm:        0.00923732 +/- 7.62e-05 (0.83%)  == '2.0000000*sigma'
    center:      0.03531364 +/- 3.07e-05 (0.09%) (init= 0.0081)
    amplitude:  -0.00012845 +/- 8.21e-07 (0.64%) (init=-0.001)



In [40]:
out_kalib_hf_4_4_peak_2.plot()
plt.show()

In [41]:
plt.plot(peak_4_4_kalib_2[0],kalib_hf_4_4_peak_2_new)
plt.plot(np.arange(0,0.052,0.00001),out_kalib_hf_4_4_peak_2.eval(x=np.arange(0,0.052,0.00001)))
plt.show()

#### Peak 3

In [42]:
kalib_hf_4_4_peak_3_new = peak_4_4_kalib_3[1]-untergrund_hf_4_4

In [43]:
plt.plot(peak_4_4_kalib_3[0],kalib_hf_4_4_peak_3_new, ' .')
plt.show()

In [44]:
mask_kalib_hf_4_4_peak_3 = np.where(peak_4_4_kalib_3[0][np.where(peak_4_4_kalib_3[0] < 0.0469)[0]] > 0.04)[0]

In [45]:
mod_kalib_hf_4_4_peak_3 = LorentzianModel()
mod_kalib_const_hf_4_4_peak_3 = ConstantModel()

In [46]:
pars_kalib_hf_4_4_peak_3 = mod_kalib_hf_4_4_peak_3.guess(kalib_hf_4_4_peak_3_new[mask_kalib_hf_4_4_peak_3],
                                                         x=peak_4_4_kalib_3[0][mask_kalib_hf_4_4_peak_3])
pars_kalib_hf_4_4_peak_3["sigma"].set(0.001)
pars_kalib_hf_4_4_peak_3["center"].set(0.0081)
pars_kalib_hf_4_4_peak_3["amplitude"].set(-0.001)

In [47]:
out_kalib_hf_4_4_peak_3 = mod_kalib_hf_4_4_peak_3.fit(kalib_hf_4_4_peak_3_new[mask_kalib_hf_4_4_peak_3],
                                                      params=pars_kalib_hf_4_4_peak_3,
                                                      x=peak_4_4_kalib_3[0][mask_kalib_hf_4_4_peak_3],
                                                      weights=1/(kalib_hf_4_4_peak_3_new[mask_kalib_hf_4_4_peak_3]*0.02))

In [48]:
print(out_kalib_hf_4_4_peak_3.fit_report(show_correl=False))

[[Model]]
    Model(lorentzian)
[[Fit Statistics]]
    # function evals   = 824
    # data points      = 275
    # variables        = 3
    chi-square         = 24.937
    reduced chi-square = 0.092
[[Variables]]
    sigma:       0.00822612 +/- 3.32e-05 (0.40%) (init= 0.001)
    fwhm:        0.01645225 +/- 6.64e-05 (0.40%)  == '2.0000000*sigma'
    center:      0.04352171 +/- 6.87e-06 (0.02%) (init= 0.0081)
    amplitude:  -0.00022417 +/- 8.13e-07 (0.36%) (init=-0.001)



In [49]:
out_kalib_hf_4_4_peak_3.plot()
plt.show()

In [50]:
plt.plot(peak_4_4_kalib_3[0],kalib_hf_4_4_peak_3_new)
plt.plot(np.arange(0,0.052,0.00001),out_kalib_hf_4_4_peak_3.eval(x=np.arange(0,0.052,0.00001)))
plt.show()

#### Linearer Fit

In [51]:
peak_4_4_time = []
peak_4_4_time.append(out_kalib_hf_4_4_peak_1.best_values["center"])
peak_4_4_time.append(out_kalib_hf_4_4_peak_2.best_values["center"])
peak_4_4_time.append(out_kalib_hf_4_4_peak_3.best_values["center"])
peak_4_4_time = np.array(peak_4_4_time)

In [52]:
peak_4_4_wellenzahl = np.array([24.130,24.145,24.149])

In [53]:
mod_peak_4_4_lin = LinearModel()

In [54]:
pars_peak_4_4_lin = mod_peak_4_4_lin.guess(peak_4_4_wellenzahl,x=peak_4_4_time)

In [55]:
out_peak_4_4_lin = mod_peak_4_4_lin.fit(peak_4_4_wellenzahl,
                                        params=pars_peak_4_4_lin,
                                        x=peak_4_4_time,
                                        weights=1/0.0003)

In [56]:
print(out_peak_4_4_lin.fit_report())

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # function evals   = 6
    # data points      = 3
    # variables        = 2
    chi-square         = 0.932
    reduced chi-square = 0.932
[[Variables]]
    slope:       0.53601116 +/- 0.010961 (2.05%) (init= 0.5360111)
    intercept:   24.1258457 +/- 0.000358 (0.00%) (init= 24.12585)
[[Correlations]] (unreported correlations are <  0.100)
    C(slope, intercept)          = -0.884 



In [57]:
out_peak_4_4_lin.plot()
plt.gca().get_yaxis().get_major_formatter().set_useOffset(False)
plt.legend(loc="best")
plt.show()

In [58]:
plt.errorbar(peak_4_4_time,peak_4_4_wellenzahl,yerr=0.0003, fmt=' .')
plt.plot(np.arange(0,0.05,0.0001),out_peak_4_4_lin.eval(x=np.arange(0,0.05,0.0001)))
plt.show()

## Fit der Hyperfeinstruckture

In [59]:
plt.plot(peak_4_4[0],peak_4_4[1]-untergrund_hf_4_4, ' .')
plt.show()

In [60]:
mask_peak_4_4_hf_1 = np.where(peak_4_4[0][np.where(peak_4_4[0] < 0.0215)[0]] > 0.021)[0]
mask_peak_4_4_hf_2 = np.where(peak_4_4[0][np.where(peak_4_4[0] < 0.023)[0]] > 0.0215)[0]
mask_peak_4_4_hf_3 = np.where(peak_4_4[0][np.where(peak_4_4[0] < 0.027)[0]] > 0.024)[0]
mask_peak_4_4_hf_4 = np.where(peak_4_4[0][np.where(peak_4_4[0] < 0.030)[0]] > 0.029)[0]
mask_peak_4_4_hf_5 = np.where(peak_4_4[0][np.where(peak_4_4[0] < 0.032)[0]] > 0.030)[0]

In [61]:
mod_peak_4_4_hf_lorenz_1 = LorentzianModel(prefix="peak_4_4_hf_1")
mod_peak_4_4_hf_lorenz_2 = LorentzianModel(prefix="peak_4_4_hf_2")
mod_peak_4_4_hf_lorenz_3 = LorentzianModel(prefix="peak_4_4_hf_3")
mod_peak_4_4_hf_lorenz_4 = LorentzianModel(prefix="peak_4_4_hf_4")
mod_peak_4_4_hf_lorenz_5 = LorentzianModel(prefix="peak_4_4_hf_5")

mod_peak_4_4_hf = mod_peak_4_4_hf_lorenz_1 + mod_peak_4_4_hf_lorenz_2 + mod_peak_4_4_hf_lorenz_3 + mod_peak_4_4_hf_lorenz_4 + mod_peak_4_4_hf_lorenz_5

In [62]:
pars_mod_peak_4_4_hf = mod_peak_4_4_hf_lorenz_1.guess(peak_4_4[1][mask_peak_4_4_hf_1],x=peak_4_4[0][mask_peak_4_4_hf_1])
pars_mod_peak_4_4_hf += mod_peak_4_4_hf_lorenz_2.guess(peak_4_4[1][mask_peak_4_4_hf_2],x=peak_4_4[0][mask_peak_4_4_hf_2])
pars_mod_peak_4_4_hf += mod_peak_4_4_hf_lorenz_3.guess(peak_4_4[1][mask_peak_4_4_hf_3],x=peak_4_4[0][mask_peak_4_4_hf_3])
pars_mod_peak_4_4_hf += mod_peak_4_4_hf_lorenz_4.guess(peak_4_4[1][mask_peak_4_4_hf_4],x=peak_4_4[0][mask_peak_4_4_hf_4])
pars_mod_peak_4_4_hf += mod_peak_4_4_hf_lorenz_5.guess(peak_4_4[1][mask_peak_4_4_hf_5],x=peak_4_4[0][mask_peak_4_4_hf_5])

In [63]:
pars_mod_peak_4_4_hf["peak_4_4_hf_1center"].set(0.0212002,vary=False)
pars_mod_peak_4_4_hf["peak_4_4_hf_2center"].set(0.022102,vary=False)
pars_mod_peak_4_4_hf["peak_4_4_hf_3center"].set(0.0259604,vary=False)
pars_mod_peak_4_4_hf["peak_4_4_hf_4center"].set(0.0298805,vary=False)
pars_mod_peak_4_4_hf["peak_4_4_hf_5center"].set(0.0310505,vary=False)
pars_mod_peak_4_4_hf["peak_4_4_hf_5amplitude"].set(-0.0000001)
pars_mod_peak_4_4_hf["peak_4_4_hf_5sigma"].set(0.001)

In [64]:
out_peak_4_4_hf = mod_peak_4_4_hf.fit(peak_4_4[1]-untergrund_hf_4_4,x=peak_4_4[0],params=pars_mod_peak_4_4_hf,weights=1/0.00001)

In [65]:
print(out_peak_4_4_hf.fit_report(show_correl=False))

[[Model]]
    ((((Model(lorentzian, prefix='peak_4_4_hf_1') + Model(lorentzian, prefix='peak_4_4_hf_2')) + Model(lorentzian, prefix='peak_4_4_hf_3')) + Model(lorentzian, prefix='peak_4_4_hf_4')) + Model(lorentzian, prefix='peak_4_4_hf_5'))
[[Fit Statistics]]
    # function evals   = 691
    # data points      = 2046
    # variables        = 10
    chi-square         = 46154.570
    reduced chi-square = 22.669
[[Variables]]
    peak_4_4_hf_1sigma:       0.00023476 +/- 6.64e-05 (28.29%) (init= 0.0002125)
    peak_4_4_hf_1fwhm:        0.00046953 +/- 0.000133 (28.29%)  == '2.0000000*peak_4_4_hf_1sigma'
    peak_4_4_hf_1center:      0.0212002 (fixed)
    peak_4_4_hf_1amplitude:  -7.7526e-08 +/- 1.96e-08 (25.30%) (init= 6.375e-08)
    peak_4_4_hf_2sigma:       0.00064116 +/- 9.30e-05 (14.50%) (init= 0.00035)
    peak_4_4_hf_2fwhm:        0.00128232 +/- 0.000186 (14.50%)  == '2.0000000*peak_4_4_hf_2sigma'
    peak_4_4_hf_2center:      0.022102 (fixed)
    peak_4_4_hf_2amplitude:  -2.7492e-07 

In [66]:
out_peak_4_4_hf.plot()
plt.show()

In [935]:
plt.errorbar(peak_4_4[0],peak_4_4[1]-untergrund_hf_4_4,yerr=0.000003, fmt=' .')
plt.plot(np.arange(0,0.05,0.0001),out_peak_4_4_hf.eval(x=np.arange(0,0.05,0.0001)))
plt.gca().get_yaxis().get_major_formatter().set_useOffset(False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlim(0,0.05)
#plt.ylim(-0.05,45)
plt.xlabel(r'Time/s', fontsize=24)
plt.ylabel("Current/V", fontsize=24)
plt.legend(loc='best', fontsize=24)
plt.show()

/home/frederik/anaconda3/lib/python3.5/site-packages/matplotlib/axes/_axes.py:519: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


In [876]:
d_aussen_l = out_peak_4_4_hf.best_values["peak_4_4_hf_1center"] - out_peak_4_4_hf.best_values["peak_4_4_hf_3center"]
d_innen_l = out_peak_4_4_hf.best_values["peak_4_4_hf_2center"] - out_peak_4_4_hf.best_values["peak_4_4_hf_3center"]
d_aussen_r = out_peak_4_4_hf.best_values["peak_4_4_hf_5center"] - out_peak_4_4_hf.best_values["peak_4_4_hf_3center"]
d_innen_r = out_peak_4_4_hf.best_values["peak_4_4_hf_4center"] - out_peak_4_4_hf.best_values["peak_4_4_hf_3center"]

In [882]:
print((d_aussen_l*out_peak_4_4_lin.best_values["slope"]*1000)/0.6)
print((d_innen_l*out_peak_4_4_lin.best_values["slope"]*1000)/0.477)
print((d_aussen_r*out_peak_4_4_lin.best_values["slope"]*1000)/0.6)
print((d_innen_r*out_peak_4_4_lin.best_values["slope"]*1000)/0.477)

-4.25253394548
-4.33573479029
4.54725075331
4.4050678912


In [879]:
2.81/1.72

1.6337209302325582

In [ ]:
4.25
4.33
4.54
4.40

## Bestimmen des Quadupolmoments

# HF für den 3 3 Peak

In [825]:
peak_3_3 = np.genfromtxt("Frederik_Henrik/Messung_2/H_F_3_3.tab", skip_header=1).T
peak_3_3_kalib_1 = np.genfromtxt("Frederik_Henrik/Messung_2/H_F_3_3_25_370.tab", skip_header=1).T
peak_3_3_kalib_2 = np.genfromtxt("Frederik_Henrik/Messung_2/H_F_3_3_25_381.tab", skip_header=1).T
peak_3_3_kalib_3 = np.genfromtxt("Frederik_Henrik/Messung_2/H_F_3_3_25_390.tab", skip_header=1).T

In [826]:
plt.plot(peak_3_3[0],peak_3_3[1],' .')
plt.show()

## Untergrund  3 3

In [827]:
submask_untergrund_hf_3_3_upper = np.where(peak_3_3[0] > 0.0265, True, False)
submask_untergrund_hf_3_3_lower = np.where(peak_3_3[0] < 0.014, True,False)
mask_untergrund_hf_3_3 = np.where(np.logical_or(submask_untergrund_hf_3_3_upper,
                                                 submask_untergrund_hf_3_3_lower))

In [828]:
mod_underground_hf_3_3 = PolynomialModel(6)

In [829]:
pars_underground_hf_3_3 = mod_underground_hf_3_3.guess(peak_3_3[1][mask_untergrund_hf_3_3],
                                                         x=peak_3_3[0][mask_untergrund_hf_3_3])

In [830]:
out_underground_hf_3_3 = mod_underground_hf_3_3.fit(peak_3_3[1][mask_untergrund_hf_3_3],
                                                      params=pars_underground_hf_3_3,
                                                      x=peak_3_3[0][mask_untergrund_hf_3_3],
                                                      weights=1/0.0001)

In [831]:
print(out_underground_hf_3_3.fit_report(show_correl=False))

[[Model]]
    Model(polynomial)
[[Fit Statistics]]
    # function evals   = 10
    # data points      = 1545
    # variables        = 7
    chi-square         = 201.672
    reduced chi-square = 0.131
[[Variables]]
    c0:   0.00183593 +/- 5.81e-06 (0.32%) (init= 0.001835934)
    c1:   0.45561181 +/- 0.003350 (0.74%) (init= 0.4556118)
    c2:  -51.9200784 +/- 0.584156 (1.13%) (init=-51.92008)
    c3:   1884.54728 +/- 42.71795 (2.27%) (init= 1884.547)
    c4:  -26780.7082 +/- 1.51e+03 (5.65%) (init=-26780.71)
    c5:   97902.1210 +/- 2.57e+04 (26.26%) (init= 97902.12)
    c6:   4.9182e+05 +/- 1.67e+05 (34.01%) (init= 491816.2)



In [832]:
out_underground_hf_3_3.plot()
plt.show()

In [843]:
plt.errorbar(peak_3_3[0],peak_3_3[1],yerr=0.00001, fmt=' .', label="Data")
plt.plot(np.arange(0,0.052,0.001),out_underground_hf_3_3.eval(x=np.arange(0,0.052,0.001)),label="Fit")
plt.gca().get_yaxis().get_major_formatter().set_useOffset(False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlim(0,0.05)
#plt.ylim(-0.05,45)
plt.xlabel(r'Time/s', fontsize=24)
plt.ylabel("Current/V", fontsize=24)
plt.legend(loc='best', fontsize=24)
plt.show()

In [834]:
untergrund_hf_3_3 = out_underground_hf_3_3.eval(x=peak_3_3[0])

In [835]:
plt.plot(peak_3_3[0],peak_3_3[1]-untergrund_hf_3_3, ' .')
plt.show()

# Kalibrierung

## Peak 1

In [844]:
kalib_hf_3_3_peak_1_new = peak_3_3_kalib_1[1]-untergrund_hf_3_3

In [845]:
plt.plot(peak_3_3_kalib_1[0],kalib_hf_3_3_peak_1_new-kalib_hf_3_3_peak_1_new.max(), ' .')
plt.show()

In [846]:
mask_kalib_hf_3_3_peak_1 = np.where(peak_3_3_kalib_1[0][np.where(peak_3_3_kalib_1[0] < 0.0117)[0]] > 0.006)[0]

In [847]:
mod_kalib_hf_3_3_peak_1 = LorentzianModel()
mod_kalib_const_hf_3_3_peak_1 = ConstantModel()

mod_kalib_hf_3_3_peak_1_composit = mod_kalib_const_hf_3_3_peak_1 +  mod_kalib_hf_3_3_peak_1

In [848]:
pars_kalib_hf_3_3_peak_1 = mod_kalib_hf_4_4_peak_1.guess(kalib_hf_3_3_peak_1_new[mask_kalib_hf_3_3_peak_1]-kalib_hf_3_3_peak_1_new.max(),
                                                         x=peak_3_3_kalib_1[0][mask_kalib_hf_3_3_peak_1])
pars_kalib_hf_3_3_peak_1 += mod_kalib_const_hf_3_3_peak_1.make_params(c=0.1)
pars_kalib_hf_3_3_peak_1["sigma"].set(0.006)
pars_kalib_hf_3_3_peak_1["center"].set(0.00889355, vary=True)
pars_kalib_hf_3_3_peak_1["amplitude"].set(-0.001)

In [849]:
out_kalib_hf_3_3_peak_1 = mod_kalib_hf_3_3_peak_1_composit.fit(kalib_hf_3_3_peak_1_new[mask_kalib_hf_3_3_peak_1]-kalib_hf_3_3_peak_1_new.max(),
                                                      params=pars_kalib_hf_3_3_peak_1,
                                                      x=peak_3_3_kalib_1[0][mask_kalib_hf_3_3_peak_1],
                                                      weights=1/(kalib_hf_3_3_peak_1_new[mask_kalib_hf_3_3_peak_1]*0.02))

In [850]:
print(out_kalib_hf_3_3_peak_1.fit_report(show_correl=False))

[[Model]]
    (Model(constant) + Model(lorentzian))
[[Fit Statistics]]
    # function evals   = 151
    # data points      = 227
    # variables        = 4
    chi-square         = 35.471
    reduced chi-square = 0.159
[[Variables]]
    sigma:       0.00427150 +/- 0.000126 (2.96%) (init= 0.006)
    fwhm:        0.00854300 +/- 0.000253 (2.96%)  == '2.0000000*sigma'
    center:      0.00887403 +/- 3.65e-06 (0.04%) (init= 0.00889355)
    amplitude:  -0.00013146 +/- 9.44e-06 (7.18%) (init=-0.001)
    c:          -0.00217082 +/- 0.000421 (19.38%) (init= 0.1)



In [867]:
plt.plot(peak_3_3_kalib_1[0],kalib_hf_3_3_peak_1_new-kalib_hf_3_3_peak_1_new.max(),label="Data")
plt.plot(np.arange(0,0.052,0.00001),out_kalib_hf_3_3_peak_1.eval(x=np.arange(0,0.052,0.00001)), label="Fit")
plt.gca().get_yaxis().get_major_formatter().set_useOffset(False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlim(0,0.05)
#plt.ylim(-0.05,45)
plt.xlabel(r'Time/s', fontsize=24)
plt.ylabel("Current/V", fontsize=24)
plt.legend(loc='best', fontsize=24)
plt.show()

## Peak 2

In [87]:
kalib_hf_3_3_peak_2_new = peak_3_3_kalib_2[1]-untergrund_hf_3_3

In [88]:
plt.plot(peak_3_3_kalib_2[0],kalib_hf_3_3_peak_2_new, ' .')
plt.show()

In [89]:
mask_kalib_hf_3_3_peak_2 = np.where(peak_3_3_kalib_2[0][np.where(peak_3_3_kalib_2[0] < 0.027)[0]] > 0.022)[0]

In [90]:
mod_kalib_hf_3_3_peak_2 = LorentzianModel()
mod_kalib_const_hf_3_3_peak_2 = ConstantModel()
mod_kalib_hf_3_3_peak_2_composit = mod_kalib_hf_3_3_peak_2 + mod_kalib_const_hf_3_3_peak_2

In [91]:
pars_kalib_hf_3_3_peak_2 = mod_kalib_hf_3_3_peak_2.guess(kalib_hf_3_3_peak_2_new[mask_kalib_hf_3_3_peak_2],
                                                         x=peak_3_3_kalib_2[0][mask_kalib_hf_3_3_peak_2])
pars_kalib_hf_3_3_peak_2 += mod_kalib_const_hf_3_3_peak_2.make_params()
pars_kalib_hf_3_3_peak_1["sigma"].set(0.001)
pars_kalib_hf_3_3_peak_1["center"].set(0.0081)
pars_kalib_hf_3_3_peak_1["amplitude"].set(-0.001)
pars_kalib_hf_3_3_peak_2["c"].set(0.1)

In [92]:
out_kalib_hf_3_3_peak_2 = mod_kalib_hf_3_3_peak_2_composit.fit(kalib_hf_3_3_peak_2_new[mask_kalib_hf_3_3_peak_2],
                                                      params=pars_kalib_hf_3_3_peak_2,
                                                      x=peak_3_3_kalib_2[0][mask_kalib_hf_3_3_peak_2],
                                                      weights=1/(kalib_hf_3_3_peak_2_new[mask_kalib_hf_3_3_peak_2]*0.02))

In [93]:
print(out_kalib_hf_3_3_peak_2.fit_report(show_correl=False))

[[Model]]
    (Model(lorentzian) + Model(constant))
[[Fit Statistics]]
    # function evals   = 236
    # data points      = 199
    # variables        = 4
    chi-square         = 16.462
    reduced chi-square = 0.084
[[Variables]]
    sigma:       0.00552522 +/- 0.000298 (5.40%) (init= 0.002475)
    fwhm:        0.01105044 +/- 0.000597 (5.40%)  == '2.0000000*sigma'
    center:      0.02484858 +/- 5.22e-06 (0.02%) (init= 0.02358125)
    amplitude:  -0.00017000 +/- 2.44e-05 (14.37%) (init= 1.291371e-05)
    c:           0.00079675 +/- 0.000883 (110.87%) (init= 0.1)



In [94]:
plt.plot(peak_3_3_kalib_2[0],kalib_hf_3_3_peak_2_new)
plt.plot(np.arange(0,0.052,0.00001),out_kalib_hf_3_3_peak_2.eval(x=np.arange(0,0.052,0.00001)))
plt.show()

## Peak 3

In [95]:
kalib_hf_3_3_peak_3_new = peak_3_3_kalib_3[1]-untergrund_hf_3_3

In [96]:
plt.plot(peak_3_3_kalib_3[0],kalib_hf_3_3_peak_3_new, ' .')
plt.show()

In [97]:
mask_kalib_hf_3_3_peak_3 = np.where(peak_3_3_kalib_3[0][np.where(peak_3_3_kalib_3[0] < 0.046)[0]] > 0.035)[0]

In [98]:
mod_kalib_hf_3_3_peak_3 = LorentzianModel()
mod_kalib_const_hf_3_3_peak_3 = ConstantModel()

In [99]:
pars_kalib_hf_3_3_peak_3 = mod_kalib_hf_3_3_peak_3.guess(kalib_hf_3_3_peak_3_new[mask_kalib_hf_3_3_peak_3],
                                                         x=peak_3_3_kalib_3[0][mask_kalib_hf_3_3_peak_3])
pars_kalib_hf_3_3_peak_3["sigma"].set(0.001)
pars_kalib_hf_3_3_peak_3["center"].set(0.0081)
pars_kalib_hf_3_3_peak_3["amplitude"].set(-0.001)

In [100]:
out_kalib_hf_3_3_peak_3 = mod_kalib_hf_3_3_peak_3.fit(kalib_hf_3_3_peak_3_new[mask_kalib_hf_3_3_peak_3],
                                                      params=pars_kalib_hf_3_3_peak_3,
                                                      x=peak_3_3_kalib_3[0][mask_kalib_hf_3_3_peak_3],
                                                      weights=1/(kalib_hf_3_3_peak_3_new[mask_kalib_hf_3_3_peak_3]*0.02))

In [101]:
print(out_kalib_hf_3_3_peak_3.fit_report(show_correl=False))

[[Model]]
    Model(lorentzian)
[[Fit Statistics]]
    # function evals   = 542
    # data points      = 439
    # variables        = 3
    chi-square         = 93.092
    reduced chi-square = 0.214
[[Variables]]
    sigma:       0.00557274 +/- 7.77e-06 (0.14%) (init= 0.001)
    fwhm:        0.01114548 +/- 1.55e-05 (0.14%)  == '2.0000000*sigma'
    center:      0.04065982 +/- 3.28e-06 (0.01%) (init= 0.0081)
    amplitude:  -0.00016156 +/- 1.49e-07 (0.09%) (init=-0.001)



In [102]:
plt.plot(peak_3_3_kalib_3[0],kalib_hf_3_3_peak_3_new)
plt.plot(np.arange(0,0.052,0.00001),out_kalib_hf_3_3_peak_3.eval(x=np.arange(0,0.052,0.00001)))
plt.show()

## Linearer Fit

In [103]:
peak_3_3_time = []
peak_3_3_time.append(out_kalib_hf_3_3_peak_1.best_values["center"])
peak_3_3_time.append(out_kalib_hf_3_3_peak_2.best_values["center"])
peak_3_3_time.append(out_kalib_hf_3_3_peak_3.best_values["center"])
peak_3_3_time = np.array(peak_3_3_time)
peak_3_3_wellenzahl = np.array([25.370,25.381,25.390])

In [104]:
mod_peak_3_3_lin = LinearModel()

In [105]:
pars_peak_3_3_lin = mod_peak_3_3_lin.guess(peak_3_3_wellenzahl,x=peak_3_3_time)


In [106]:
out_peak_3_3_lin = mod_peak_3_3_lin.fit(peak_3_3_wellenzahl,
                                        params=pars_peak_3_3_lin,
                                        x=peak_3_3_time,
                                        weights=1/0.0005)

In [107]:
print(out_peak_3_3_lin.fit_report())

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # function evals   = 6
    # data points      = 3
    # variables        = 2
    chi-square         = 2.400
    reduced chi-square = 2.400
[[Variables]]
    slope:       0.62931458 +/- 0.034460 (5.48%) (init= 0.6293144)
    intercept:   25.3647300 +/- 0.000964 (0.00%) (init= 25.36473)
[[Correlations]] (unreported correlations are <  0.100)
    C(slope, intercept)          = -0.886 



In [108]:
out_peak_3_3_lin.plot()
plt.gca().get_yaxis().get_major_formatter().set_useOffset(False)
plt.legend(loc="best")
plt.show()

In [872]:
plt.errorbar(peak_3_3_time,peak_3_3_wellenzahl,yerr=0.00035, fmt=' .',label="Data")
plt.plot(np.arange(0,0.05,0.0001),out_peak_3_3_lin.eval(x=np.arange(0,0.05,0.0001)),label="Fit")
plt.gca().get_yaxis().get_major_formatter().set_useOffset(False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlim(0,0.05)
#plt.ylim(-0.05,45)
plt.xlabel(r'Time/s', fontsize=24)
plt.ylabel("Frequency/GHz", fontsize=24)
plt.legend(loc='best', fontsize=24)
plt.show()

# Peaks anfitten

In [320]:
plt.plot(peak_3_3[0],peak_3_3[1]-untergrund_hf_3_3, ' .')
plt.show()

In [321]:
mask_peak_3_3_hf_1 = np.where(peak_3_3[0][np.where(peak_3_3[0] < 0.0167)[0]] > 0.015)[0]
mask_peak_3_3_hf_2 = np.where(peak_3_3[0][np.where(peak_3_3[0] < 0.0176)[0]] > 0.0167)[0]
mask_peak_3_3_hf_3 = np.where(peak_3_3[0][np.where(peak_3_3[0] < 0.0224)[0]] > 0.0176)[0]
mask_peak_3_3_hf_4 = np.where(peak_3_3[0][np.where(peak_3_3[0] < 0.0235)[0]] > 0.0224)[0]
mask_peak_3_3_hf_5 = np.where(peak_3_3[0][np.where(peak_3_3[0] < 0.0245)[0]] > 0.0235)[0]
mask_peak_3_3_hf = np.where(peak_3_3[0][np.where(peak_3_3[0] < 0.0268)[0]] > 0.012)[0]

In [322]:
mod_peak_3_3_hf_lorenz_1 = LorentzianModel(prefix="peak_3_3_hf_1")
mod_peak_3_3_hf_lorenz_2 = LorentzianModel(prefix="peak_3_3_hf_2")
mod_peak_3_3_hf_lorenz_3 = LorentzianModel(prefix="peak_3_3_hf_3")
mod_peak_3_3_hf_lorenz_4 = LorentzianModel(prefix="peak_3_3_hf_4")
mod_peak_3_3_hf_lorenz_5 = LorentzianModel(prefix="peak_3_3_hf_5")

mod_peak_3_3_hf = mod_peak_3_3_hf_lorenz_1 + mod_peak_3_3_hf_lorenz_2 + mod_peak_3_3_hf_lorenz_3 + mod_peak_3_3_hf_lorenz_4 + mod_peak_3_3_hf_lorenz_5

In [323]:
pars_mod_peak_3_3_hf = mod_peak_3_3_hf_lorenz_1.guess(peak_3_3[1][mask_peak_3_3_hf_1],x=peak_3_3[0][mask_peak_3_3_hf_1])
pars_mod_peak_3_3_hf += mod_peak_3_3_hf_lorenz_2.guess(peak_3_3[1][mask_peak_3_3_hf_2],x=peak_3_3[0][mask_peak_3_3_hf_2])
pars_mod_peak_3_3_hf += mod_peak_3_3_hf_lorenz_3.guess(peak_3_3[1][mask_peak_3_3_hf_3],x=peak_3_3[0][mask_peak_3_3_hf_3])
pars_mod_peak_3_3_hf += mod_peak_3_3_hf_lorenz_4.guess(peak_3_3[1][mask_peak_3_3_hf_4],x=peak_3_3[0][mask_peak_3_3_hf_4])
pars_mod_peak_3_3_hf += mod_peak_3_3_hf_lorenz_5.guess(peak_3_3[1][mask_peak_3_3_hf_5],x=peak_3_3[0][mask_peak_3_3_hf_5])

In [333]:
pars_mod_peak_3_3_hf["peak_3_3_hf_1center"].set(0.0161390,vary=True)
pars_mod_peak_3_3_hf["peak_3_3_hf_2center"].set(0.017338,vary=True)
pars_mod_peak_3_3_hf["peak_3_3_hf_3center"].set(0.0199876,vary=True)
pars_mod_peak_3_3_hf["peak_3_3_hf_4center"].set(0.0229625,vary=True)
pars_mod_peak_3_3_hf["peak_3_3_hf_5center"].set(0.0239799,vary=True)
pars_mod_peak_3_3_hf["peak_3_3_hf_1amplitude"].set(-0.0000005)
pars_mod_peak_3_3_hf["peak_3_3_hf_2amplitude"].set(-0.000001)
pars_mod_peak_3_3_hf["peak_3_3_hf_3amplitude"].set(-0.00001)
pars_mod_peak_3_3_hf["peak_3_3_hf_4amplitude"].set(-0.000001)
pars_mod_peak_3_3_hf["peak_3_3_hf_5amplitude"].set(-0.0000005)
pars_mod_peak_3_3_hf["peak_3_3_hf_1sigma"].set(0.0001)
pars_mod_peak_3_3_hf["peak_3_3_hf_2sigma"].set(0.0001)
pars_mod_peak_3_3_hf["peak_3_3_hf_3sigma"].set(0.0001)
pars_mod_peak_3_3_hf["peak_3_3_hf_4sigma"].set(0.0001)
pars_mod_peak_3_3_hf["peak_3_3_hf_5sigma"].set(0.0001)

In [345]:
out_peak_3_3_hf = mod_peak_3_3_hf.fit((peak_3_3[1]-untergrund_hf_3_3)[mask_peak_3_3_hf],x=peak_3_3[0][mask_peak_3_3_hf],params=pars_mod_peak_3_3_hf,weights=1/0.00004)

In [346]:
print(out_peak_3_3_hf.fit_report(show_correl=False))

[[Model]]
    ((((Model(lorentzian, prefix='peak_3_3_hf_1') + Model(lorentzian, prefix='peak_3_3_hf_2')) + Model(lorentzian, prefix='peak_3_3_hf_3')) + Model(lorentzian, prefix='peak_3_3_hf_4')) + Model(lorentzian, prefix='peak_3_3_hf_5'))
[[Fit Statistics]]
    # function evals   = 309
    # data points      = 591
    # variables        = 15
    chi-square         = 539.523
    reduced chi-square = 0.937
[[Variables]]
    peak_3_3_hf_1sigma:       0.00040488 +/- 3.19e-05 (7.88%) (init= 0.0001)
    peak_3_3_hf_1fwhm:        0.00080975 +/- 6.38e-05 (7.88%)  == '2.0000000*peak_3_3_hf_1sigma'
    peak_3_3_hf_1center:      0.01608357 +/- 1.67e-05 (0.10%) (init= 0.016139)
    peak_3_3_hf_1amplitude:  -4.4009e-07 +/- 4.42e-08 (10.05%) (init=-5e-07)
    peak_3_3_hf_2sigma:       0.00077106 +/- 4.03e-05 (5.23%) (init= 0.0001)
    peak_3_3_hf_2fwhm:        0.00154212 +/- 8.07e-05 (5.23%)  == '2.0000000*peak_3_3_hf_2sigma'
    peak_3_3_hf_2center:      0.01721402 +/- 2.51e-05 (0.15%) (init= 0.01

In [347]:
out_peak_3_3_hf.plot()
plt.show()

In [936]:
plt.errorbar(peak_3_3[0],peak_3_3[1]-untergrund_hf_3_3,yerr=0.000003, fmt=' .r')
plt.plot(np.arange(0,0.05,0.00001),out_peak_3_3_hf.eval(x=np.arange(0,0.05,0.00001)))
plt.gca().get_yaxis().get_major_formatter().set_useOffset(False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlim(0,0.05)
#plt.ylim(-0.05,45)
plt.xlabel(r'Time/s', fontsize=24)
plt.ylabel("Current/V", fontsize=24)
plt.legend(loc='best', fontsize=24)
plt.show()

/home/frederik/anaconda3/lib/python3.5/site-packages/matplotlib/axes/_axes.py:519: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


In [868]:
d_aussen_l = out_peak_3_3_hf.best_values["peak_3_3_hf_1center"] - out_peak_3_3_hf.best_values["peak_3_3_hf_3center"]
d_innen_l = out_peak_3_3_hf.best_values["peak_3_3_hf_2center"] - out_peak_3_3_hf.best_values["peak_3_3_hf_3center"]
d_aussen_r = out_peak_3_3_hf.best_values["peak_3_3_hf_5center"] - out_peak_3_3_hf.best_values["peak_3_3_hf_3center"]
d_innen_r = out_peak_3_3_hf.best_values["peak_3_3_hf_4center"] - out_peak_3_3_hf.best_values["peak_3_3_hf_3center"]

In [898]:
print((d_aussen_l*out_peak_3_3_lin.best_values["slope"]*1000)/0.566)
print((d_innen_l*out_peak_3_3_lin.best_values["slope"]*1000)/0.417)
print((d_aussen_r*out_peak_3_3_lin.best_values["slope"]*1000)/0.566)
print((d_innen_r*out_peak_3_3_lin.best_values["slope"]*1000)/0.417)

-5.29269128555
-5.82289539242
5.65949496084
5.91600980401


In [885]:
1.95/0.417

4.676258992805756

In [899]:
(5.29 + 5.65 + 4.54 + 4.33)/4

4.952500000000001

In [901]:
(5.82 + 5.91 + 4.40 + 4.25)/4

5.095000000000001

In [ ]:
5.29269128555 + 5.82289539242 + 5.65949496084 + 5.91600980401

In [903]:
54**0.5

7.3484692283495345

In [904]:
23**0.5

4.795831523312719

In [905]:
(4.95 + 5.09)/2

5.02

In [906]:
(25+49)**0.5

8.602325267042627

In [910]:
1-4.14/5.02

0.17529880478087645

In [909]:
5.02/4.14

1.21256038647343

# HF für den 6 6 Peak

In [349]:
peak_6_6 = np.genfromtxt("Frederik_Henrik/Messung_2/H_F_6_6.tab", skip_header=1).T
peak_6_6_kalib_1 = np.genfromtxt("Frederik_Henrik/Messung_2/H_F_6_6_25_052.tab", skip_header=1).T
peak_6_6_kalib_2 = np.genfromtxt("Frederik_Henrik/Messung_2/H_F_6_6_25_060.tab", skip_header=1).T
peak_6_6_kalib_3 = np.genfromtxt("Frederik_Henrik/Messung_2/H_F_6_6_25_070.tab", skip_header=1).T

In [352]:
plt.plot(peak_6_6[0],peak_6_6[1],' .')
plt.show()

## Untergrund 6 6

In [353]:
submask_untergrund_hf_6_6_upper = np.where(peak_6_6[0] > 0.0353226, True, False)
submask_untergrund_hf_6_6_lower = np.where(peak_6_6[0] < 0.0184839, True,False)
mask_untergrund_hf_6_6 = np.where(np.logical_or(submask_untergrund_hf_6_6_upper,
                                                 submask_untergrund_hf_6_6_lower))

In [354]:
mod_underground_hf_6_6 = PolynomialModel(6)

In [355]:
pars_underground_hf_6_6 = mod_underground_hf_6_6.guess(peak_6_6[1][mask_untergrund_hf_6_6],
                                                         x=peak_6_6[0][mask_untergrund_hf_6_6])

In [356]:
out_underground_hf_6_6 = mod_underground_hf_6_6.fit(peak_6_6[1][mask_untergrund_hf_6_6],
                                                      params=pars_underground_hf_6_6,
                                                      x=peak_6_6[0][mask_untergrund_hf_6_6],
                                                      weights=1/0.0001)

In [357]:
print(out_underground_hf_6_6.fit_report(show_correl=False))

[[Model]]
    Model(polynomial)
[[Fit Statistics]]
    # function evals   = 10
    # data points      = 1373
    # variables        = 7
    chi-square         = 76.446
    reduced chi-square = 0.056
[[Variables]]
    c0:  -0.00033811 +/- 3.92e-06 (1.16%) (init=-0.0003381117)
    c1:   0.00205214 +/- 0.002486 (121.18%) (init= 0.002052141)
    c2:   5.38350082 +/- 0.488275 (9.07%) (init= 5.383501)
    c3:  -569.830585 +/- 39.63822 (6.96%) (init=-569.8306)
    c4:   26094.3753 +/- 1.47e+03 (5.65%) (init= 26094.38)
    c5:  -5.0156e+05 +/- 2.53e+04 (5.04%) (init=-501557.7)
    c6:   3.3809e+06 +/- 1.63e+05 (4.81%) (init= 3380912)



In [358]:
out_underground_hf_6_6.plot()
plt.show()

In [359]:
plt.plot(peak_6_6[0],peak_6_6[1], ' .')
plt.plot(np.arange(0,0.052,0.001),out_underground_hf_6_6.eval(x=np.arange(0,0.052,0.001)))
plt.show()

In [360]:
untergrund_hf_6_6 = out_underground_hf_6_6.eval(x=peak_6_6[0])

In [361]:
plt.plot(peak_6_6[0],peak_6_6[1]-untergrund_hf_6_6, ' .')
plt.show()

## Kalibrierung  6 6

## Peak 1

In [377]:
kalib_hf_6_6_peak_1_new = peak_6_6_kalib_1[1]-untergrund_hf_6_6

In [364]:
plt.plot(peak_6_6_kalib_1[0],kalib_hf_6_6_peak_1_new, ' .')
plt.show()

In [365]:
mask_kalib_hf_6_6_peak_1 = np.where(peak_6_6_kalib_1[0][np.where(peak_6_6_kalib_1[0] < 0.0474194)[0]] > 0.0406452)[0]

In [366]:
mod_kalib_hf_6_6_peak_1 = LorentzianModel()
mod_kalib_const_hf_6_6_peak_1 = ConstantModel()

mod_kalib_hf_6_6_peak_1_composit = mod_kalib_const_hf_6_6_peak_1 +  mod_kalib_hf_6_6_peak_1

In [371]:
pars_kalib_hf_6_6_peak_1 = mod_kalib_hf_6_6_peak_1.guess(kalib_hf_6_6_peak_1_new[mask_kalib_hf_6_6_peak_1],
                                                         x=peak_6_6_kalib_1[0][mask_kalib_hf_6_6_peak_1])
pars_kalib_hf_6_6_peak_1 += mod_kalib_const_hf_6_6_peak_1.make_params(c=0.1)
pars_kalib_hf_6_6_peak_1["sigma"].set(0.006)
pars_kalib_hf_6_6_peak_1["center"].set(0.044, vary=True)
pars_kalib_hf_6_6_peak_1["amplitude"].set(-0.001)

In [372]:
out_kalib_hf_6_6_peak_1 = mod_kalib_hf_6_6_peak_1_composit.fit(kalib_hf_6_6_peak_1_new[mask_kalib_hf_6_6_peak_1],
                                                      params=pars_kalib_hf_6_6_peak_1,
                                                      x=peak_6_6_kalib_1[0][mask_kalib_hf_6_6_peak_1],
                                                      weights=1/(kalib_hf_6_6_peak_1_new[mask_kalib_hf_6_6_peak_1]*0.02))

In [373]:
print(out_kalib_hf_6_6_peak_1.fit_report(show_correl=False))

[[Model]]
    (Model(constant) + Model(lorentzian))
[[Fit Statistics]]
    # function evals   = 67
    # data points      = 271
    # variables        = 4
    chi-square         = 18.635
    reduced chi-square = 0.070
[[Variables]]
    sigma:       0.00471486 +/- 8.32e-05 (1.77%) (init= 0.006)
    fwhm:        0.00942973 +/- 0.000166 (1.77%)  == '2.0000000*sigma'
    center:      0.04403430 +/- 2.94e-06 (0.01%) (init= 0.044)
    amplitude:  -0.00012838 +/- 5.36e-06 (4.18%) (init=-0.001)
    c:          -0.00170224 +/- 0.000213 (12.54%) (init= 0.1)



In [411]:
plt.plot(peak_6_6_kalib_1[0],kalib_hf_6_6_peak_1_new)
plt.plot(np.arange(0,0.052,0.00001),out_kalib_hf_6_6_peak_1.eval(x=np.arange(0,0.052,0.00001)))
plt.show()

## Peak 2

In [378]:
kalib_hf_6_6_peak_2_new = peak_6_6_kalib_2[1]-untergrund_hf_6_6

In [379]:
plt.plot(peak_6_6_kalib_2[0],kalib_hf_6_6_peak_2_new, ' .')
plt.show()

In [380]:
mask_kalib_hf_6_6_peak_2 = np.where(peak_6_6_kalib_2[0][np.where(peak_6_6_kalib_2[0] < 0.0340645)[0]] > 0.0285484)[0]

In [381]:
mod_kalib_hf_6_6_peak_2 = LorentzianModel()
mod_kalib_const_hf_6_6_peak_2 = ConstantModel()

mod_kalib_hf_6_6_peak_2_composit = mod_kalib_const_hf_6_6_peak_2 +  mod_kalib_hf_6_6_peak_2

In [382]:
pars_kalib_hf_6_6_peak_2 = mod_kalib_hf_6_6_peak_2.guess(kalib_hf_6_6_peak_2_new[mask_kalib_hf_6_6_peak_2],
                                                         x=peak_6_6_kalib_2[0][mask_kalib_hf_6_6_peak_2])
pars_kalib_hf_6_6_peak_2 += mod_kalib_const_hf_6_6_peak_2.make_params(c=0.1)
pars_kalib_hf_6_6_peak_2["sigma"].set(0.006)
pars_kalib_hf_6_6_peak_2["center"].set(0.00889355, vary=True)
pars_kalib_hf_6_6_peak_2["amplitude"].set(-0.001)

In [383]:
out_kalib_hf_6_6_peak_2 = mod_kalib_hf_6_6_peak_2_composit.fit(kalib_hf_6_6_peak_2_new[mask_kalib_hf_6_6_peak_2],
                                                      params=pars_kalib_hf_6_6_peak_2,
                                                      x=peak_6_6_kalib_2[0][mask_kalib_hf_6_6_peak_2],
                                                      weights=1/(kalib_hf_6_6_peak_2_new[mask_kalib_hf_6_6_peak_2]*0.02))

In [384]:
print(out_kalib_hf_6_6_peak_2.fit_report(show_correl=False))

[[Model]]
    (Model(constant) + Model(lorentzian))
[[Fit Statistics]]
    # function evals   = 10003
    # data points      = 221
    # variables        = 4
    chi-square         = 21.725
    reduced chi-square = 0.100
[[Variables]]
    sigma:       0.05019383 +/- 0.155550 (309.90%) (init= 0.006)
    fwhm:        0.10038766 +/- 0.311101 (309.90%)  == '2.0000000*sigma'
    center:      0.03107743 +/- 4.50e-06 (0.01%) (init= 0.00889355)
    amplitude:  -0.11343642 +/- 1.053391 (928.62%) (init=-0.001)
    c:           0.70944096 +/- 4.447978 (626.97%) (init= 0.1)



In [412]:
plt.plot(peak_6_6_kalib_2[0],kalib_hf_6_6_peak_2_new)
plt.plot(np.arange(0.025,0.038,0.001),out_kalib_hf_6_6_peak_2.eval(x=np.arange(0.025,0.038,0.001)))
plt.show()

## Peak 3

In [393]:
kalib_hf_6_6_peak_3_new = peak_6_6_kalib_3[1]-untergrund_hf_6_6

In [394]:
plt.plot(peak_6_6_kalib_3[0],kalib_hf_6_6_peak_3_new-kalib_hf_6_6_peak_3_new.max(), ' .')
plt.show()

In [395]:
mask_kalib_hf_6_6_peak_3 = np.where(peak_6_6_kalib_3[0][np.where(peak_6_6_kalib_3[0] < 0.0150968)[0]] > 0.0082201)[0]

In [396]:
mod_kalib_hf_6_6_peak_3 = LorentzianModel()
mod_kalib_const_hf_6_6_peak_3 = ConstantModel()

mod_kalib_hf_6_6_peak_3_composit = mod_kalib_const_hf_6_6_peak_3 +  mod_kalib_hf_6_6_peak_3

In [399]:
pars_kalib_hf_6_6_peak_3 = mod_kalib_hf_6_6_peak_3.guess(kalib_hf_6_6_peak_3_new[mask_kalib_hf_6_6_peak_3],
                                                         x=peak_6_6_kalib_3[0][mask_kalib_hf_6_6_peak_3])
pars_kalib_hf_6_6_peak_3 += mod_kalib_const_hf_6_6_peak_3.make_params(c=0.1)
pars_kalib_hf_6_6_peak_3["sigma"].set(0.006)
pars_kalib_hf_6_6_peak_3["center"].set(0.012, vary=True)
pars_kalib_hf_6_6_peak_3["amplitude"].set(-0.001)

In [400]:
out_kalib_hf_6_6_peak_3 = mod_kalib_hf_6_6_peak_3_composit.fit(kalib_hf_6_6_peak_3_new[mask_kalib_hf_6_6_peak_3],
                                                      params=pars_kalib_hf_6_6_peak_3,
                                                      x=peak_6_6_kalib_3[0][mask_kalib_hf_6_6_peak_3],
                                                      weights=1/(kalib_hf_6_6_peak_3_new[mask_kalib_hf_6_6_peak_3]*0.02))

In [401]:
print(out_kalib_hf_6_6_peak_3.fit_report(show_correl=False))

[[Model]]
    (Model(constant) + Model(lorentzian))
[[Fit Statistics]]
    # function evals   = 43
    # data points      = 275
    # variables        = 4
    chi-square         = 27.967
    reduced chi-square = 0.103
[[Variables]]
    sigma:       0.00547477 +/- 0.000139 (2.55%) (init= 0.006)
    fwhm:        0.01094955 +/- 0.000279 (2.55%)  == '2.0000000*sigma'
    center:      0.01173960 +/- 3.58e-06 (0.03%) (init= 0.012)
    amplitude:  -0.00016056 +/- 1.01e-05 (6.31%) (init=-0.001)
    c:           5.0098e-05 +/- 0.000356 (709.76%) (init= 0.1)



In [403]:
plt.plot(peak_6_6_kalib_3[0],kalib_hf_6_6_peak_3_new)
plt.plot(np.arange(0,0.052,0.00001),out_kalib_hf_6_6_peak_3.eval(x=np.arange(0,0.052,0.00001)))
plt.show()

## Linearer Fit

In [423]:
peak_6_6_time = []
peak_6_6_time.append(out_kalib_hf_6_6_peak_1.best_values["center"])
peak_6_6_time.append(out_kalib_hf_6_6_peak_2.best_values["center"])
peak_6_6_time.append(out_kalib_hf_6_6_peak_3.best_values["center"])
peak_6_6_time = np.array(peak_6_6_time)
peak_6_6_wellenzahl = np.array([25.052,25.060,25.070])

In [424]:
mod_peak_6_6_lin = LinearModel()

In [425]:
pars_peak_6_6_lin = mod_peak_6_6_lin.guess(peak_6_6_wellenzahl,x=peak_6_6_time)

In [426]:
out_peak_6_6_lin = mod_peak_6_6_lin.fit(peak_6_6_wellenzahl,
                                        params=pars_peak_6_6_lin,
                                        x=peak_6_6_time,
                                        weights=1/0.0005)

In [427]:
print(out_peak_6_6_lin.fit_report())

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # function evals   = 6
    # data points      = 3
    # variables        = 2
    chi-square         = 1.594
    reduced chi-square = 1.594
[[Variables]]
    slope:      -0.55423369 +/- 0.027469 (4.96%) (init=-0.5542334)
    intercept:   25.0767119 +/- 0.000875 (0.00%) (init= 25.07671)
[[Correlations]] (unreported correlations are <  0.100)
    C(slope, intercept)          = -0.909 



In [428]:
out_peak_6_6_lin.plot()
plt.gca().get_yaxis().get_major_formatter().set_useOffset(False)
plt.legend(loc="best")
plt.show()

In [429]:
plt.errorbar(peak_6_6_time,peak_6_6_wellenzahl,yerr=0.00035, fmt=' .')
plt.plot(np.arange(0,0.05,0.0001),out_peak_6_6_lin.eval(x=np.arange(0,0.05,0.0001)))
plt.gca().get_yaxis().get_major_formatter().set_useOffset(False)
plt.show()

## Fit der HF

In [432]:
plt.plot(peak_6_6[0],peak_6_6[1]-untergrund_hf_6_6, ' .')
plt.show()

In [ ]:
mask_peak_6_6_hf_1 = np.where(peak_6_6[0][np.where(peak_3_3[0] < 0.0167)[0]] > 0.015)[0]
mask_peak_6_6_hf_2 = np.where(peak_6_6[0][np.where(peak_3_3[0] < 0.0176)[0]] > 0.0167)[0]
mask_peak_6_6_hf_3 = np.where(peak_6_6[0][np.where(peak_3_3[0] < 0.0224)[0]] > 0.0176)[0]
mask_peak_6_6_hf_4 = np.where(peak_6_6[0][np.where(peak_3_3[0] < 0.0235)[0]] > 0.0224)[0]
mask_peak_6_6_hf_5 = np.where(peak_6_6[0][np.where(peak_3_3[0] < 0.0245)[0]] > 0.0235)[0]
mask_peak_6_6_hf = np.where(peak_6_6[0][np.where(peak_6_6[0] < 0.0268)[0]] > 0.012)[0]

In [ ]:
mod_peak_6_6_hf_lorenz_1 = LorentzianModel(prefix="peak_6_6_hf_1")
mod_peak_6_6_hf_lorenz_2 = LorentzianModel(prefix="peak_6_6_hf_2")
mod_peak_6_6_hf_lorenz_3 = LorentzianModel(prefix="peak_6_6_hf_3")
mod_peak_6_6_hf_lorenz_4 = LorentzianModel(prefix="peak_6_6_hf_4")
mod_peak_6_6_hf_lorenz_5 = LorentzianModel(prefix="peak_6_6_hf_5")

mod_peak_6_6_hf = mod_peak_6_6_hf_lorenz_1 + mod_peak_6_6_hf_lorenz_2 + mod_peak_6_6_hf_lorenz_3 + mod_peak_6_6_hf_lorenz_4 + mod_peak_6_6_hf_lorenz_5

In [ ]:
pars_mod_peak_6_6_hf = mod_peak_3_3_hf_lorenz_1.guess(peak_6_6[1][mask_peak_6_6_hf_1],x=peak_6_6[0][mask_peak_6_6_hf_1])
pars_mod_peak_6_6_hf += mod_peak_3_3_hf_lorenz_2.guess(peak_3_3[1][mask_peak_6_6_hf_2],x=peak_6_6[0][mask_peak_6_6_hf_2])
pars_mod_peak_6_6_hf += mod_peak_3_3_hf_lorenz_3.guess(peak_3_3[1][mask_peak_6_6_hf_3],x=peak_6_6[0][mask_peak_6_6_hf_3])
pars_mod_peak_6_6_hf += mod_peak_3_3_hf_lorenz_4.guess(peak_3_3[1][mask_peak_6_6_hf_4],x=peak_6_6[0][mask_peak_6_6_hf_4])
pars_mod_peak_6_6_hf += mod_peak_3_3_hf_lorenz_5.guess(peak_3_3[1][mask_peak_6_6_hf_5],x=peak_6_6[0][mask_peak_6_6_hf_5])

In [ ]:
pars_mod_peak_6_6_hf["peak_6_6_hf_1center"].set(0.0161390,vary=True)
pars_mod_peak_6_6_hf["peak_6_6_hf_2center"].set(0.017338,vary=True)
pars_mod_peak_6_6_hf["peak_6_6_hf_3center"].set(0.0199876,vary=True)
pars_mod_peak_6_6_hf["peak_6_6_hf_4center"].set(0.0229625,vary=True)
pars_mod_peak_6_6_hf["peak_6_6_hf_5center"].set(0.0239799,vary=True)
pars_mod_peak_6_6_hf["peak_6_6_hf_1amplitude"].set(-0.0000005)
pars_mod_peak_6_6_hf["peak_6_6_hf_2amplitude"].set(-0.000001)
pars_mod_peak_6_6_hf["peak_6_6_hf_3amplitude"].set(-0.00001)
pars_mod_peak_6_6_hf["peak_6_6_hf_4amplitude"].set(-0.000001)
pars_mod_peak_6_6_hf["peak_6_6_hf_5amplitude"].set(-0.0000005)
pars_mod_peak_6_6_hf["peak_6_6_hf_1sigma"].set(0.0001)
pars_mod_peak_6_6_hf["peak_6_6_hf_2sigma"].set(0.0001)
pars_mod_peak_6_6_hf["peak_6_6_hf_3sigma"].set(0.0001)
pars_mod_peak_6_6_hf["peak_6_6_hf_4sigma"].set(0.0001)
pars_mod_peak_6_6_hf["peak_6_6_hf_5sigma"].set(0.0001)

In [ ]:
out_peak_6_6_hf = mod_peak_6_6_hf.fit((peak_6_6[1]-untergrund_hf_6_6)[mask_peak_6_6_hf],x=peak_6_6[0][mask_peak_6_6_hf],params=pars_mod_peak_6_6_hf,weights=1/0.00004)

In [ ]:
print(out_peak_3_3_hf.fit_report(show_correl=False))

In [ ]:
out_peak_6_6_hf.plot()
plt.show()

In [ ]:
plt.errorbar(peak_6_6[0],peak_6_6[1]-untergrund_hf_6_6,yerr=0.000003, fmt=' .')
plt.plot(np.arange(0,0.05,0.00001),out_peak_6_6_hf.eval(x=np.arange(0,0.05,0.00001)))
plt.show()

# Druck

In [120]:
kalib_druck_1 = np.genfromtxt("Frederik_Henrik/Messung_3/3_3_23_888_kalib_1.tab", skip_header=1).T
kalib_druck_2 = np.genfromtxt("Frederik_Henrik/Messung_3/3_3_23_880_kalib_2.tab", skip_header=1).T
kalib_druck_3 = np.genfromtxt("Frederik_Henrik/Messung_3/3_3_23_870_kalib_4.tab", skip_header=1).T

## Kalibration

### Peak 1

In [678]:
plt.plot(kalib_druck_1[0],kalib_druck_1[1])
plt.show()

In [679]:
mask_kalib_druck_1 = np.where(kalib_druck_1[0][np.where(kalib_druck_1[0] < 0.046)[0]] > 0.037)[0]

In [680]:
mod_druck_kalib_1 = LorentzianModel()

In [681]:
pars_druck_kalib_1 = mod_druck_kalib_1.guess(kalib_druck_1[1][mask_kalib_druck_1],x=kalib_druck_1[0][mask_kalib_druck_1])

In [682]:
out_druck_kalib_1 = mod_druck_kalib_1.fit(kalib_druck_1[1][mask_kalib_druck_1],
                                          params=pars_druck_kalib_1,
                                          x=kalib_druck_1[0][mask_kalib_druck_1],
                                          weights=1/(kalib_druck_1[1][mask_kalib_druck_1]*0.02))

In [683]:
print(out_druck_kalib_1.fit_report(show_correl=False))

[[Model]]
    Model(lorentzian)
[[Fit Statistics]]
    # function evals   = 40
    # data points      = 359
    # variables        = 3
    chi-square         = 342.963
    reduced chi-square = 0.963
[[Variables]]
    sigma:       0.00461452 +/- 1.52e-05 (0.33%) (init= 0.004475)
    fwhm:        0.00922904 +/- 3.04e-05 (0.33%)  == '2.0000000*sigma'
    center:      0.04101482 +/- 6.72e-06 (0.02%) (init= 0.04239009)
    amplitude:  -0.00011842 +/- 2.63e-07 (0.22%) (init= 5.012e-05)



In [684]:
plt.errorbar(kalib_druck_1[0],kalib_druck_1[1],yerr=kalib_druck_1[1]*0.05)
plt.plot(np.arange(0,0.07,0.0001),out_druck_kalib_1.eval(x=np.arange(0,0.07,0.0001)))
plt.show()

### Peak 2

In [685]:
plt.plot(kalib_druck_2[0],kalib_druck_2[1])
plt.show()

In [686]:
mask_kalib_druck_2 = np.where(kalib_druck_2[0][np.where(kalib_druck_2[0] < 0.0309)[0]] > 0.024)[0]

In [687]:
mod_druck_kalib_2 = LorentzianModel()

In [688]:
pars_druck_kalib_2 = mod_druck_kalib_2.guess(kalib_druck_2[1][mask_kalib_druck_2],x=kalib_druck_2[0][mask_kalib_druck_2])
pars_druck_kalib_2["center"].set(0.0269, vary=False)

In [689]:
out_druck_kalib_2 = mod_druck_kalib_2.fit(kalib_druck_2[1][mask_kalib_druck_2],
                                          params=pars_druck_kalib_2,
                                          x=kalib_druck_2[0][mask_kalib_druck_2],
                                          weights=1/(kalib_druck_2[1][mask_kalib_druck_2]*0.02))

In [690]:
print(out_druck_kalib_2.fit_report())

[[Model]]
    Model(lorentzian)
[[Fit Statistics]]
    # function evals   = 21
    # data points      = 275
    # variables        = 2
    chi-square         = 307.737
    reduced chi-square = 1.127
[[Variables]]
    sigma:       0.00403523 +/- 1.76e-05 (0.44%) (init= 0.003425)
    fwhm:        0.00807046 +/- 3.52e-05 (0.44%)  == '2.0000000*sigma'
    center:      0.0269 (fixed)
    amplitude:  -0.00010306 +/- 3.18e-07 (0.31%) (init= 3.562e-05)
[[Correlations]] (unreported correlations are <  0.100)
    C(sigma, amplitude)          = -0.910 



In [691]:
plt.errorbar(kalib_druck_2[0],kalib_druck_2[1],yerr=kalib_druck_2[1]*0.05)
plt.plot(np.arange(0,0.05,0.0001),out_druck_kalib_2.eval(x=np.arange(0,0.05,0.0001)))
plt.show()

### Peak 3

In [692]:
plt.plot(kalib_druck_3[0],kalib_druck_3[1])
plt.show()

In [693]:
mask_kalib_druck_3 = np.where(kalib_druck_3[0][np.where(kalib_druck_3[0] < 0.014)[0]] > 0.008)[0]

In [694]:
mod_druck_kalib_3 = LorentzianModel()

In [695]:
pars_druck_kalib_3 = mod_druck_kalib_3.guess(kalib_druck_3[1][mask_kalib_druck_3],x=kalib_druck_3[0][mask_kalib_druck_3])
#pars_druck_kalib_3["amplitude"].set(-0.00015)
pars_druck_kalib_3["center"].set(0.01125)

In [696]:
out_druck_kalib_3 = mod_druck_kalib_3.fit(kalib_druck_3[1][mask_kalib_druck_3],
                                          params=pars_druck_kalib_3,
                                          x=kalib_druck_3[0][mask_kalib_druck_3],
                                          weights=1/(kalib_druck_3[1][mask_kalib_druck_3]*0.02))

In [697]:
print(out_druck_kalib_3.fit_report())

[[Model]]
    Model(lorentzian)
[[Fit Statistics]]
    # function evals   = 27
    # data points      = 239
    # variables        = 3
    chi-square         = 266.165
    reduced chi-square = 1.128
[[Variables]]
    sigma:       0.00298962 +/- 1.26e-05 (0.42%) (init= 0.002975)
    fwhm:        0.00597924 +/- 2.53e-05 (0.42%)  == '2.0000000*sigma'
    center:      0.01103083 +/- 5.45e-06 (0.05%) (init= 0.01125)
    amplitude:  -6.2858e-05 +/- 1.75e-07 (0.28%) (init= 2.5585e-05)
[[Correlations]] (unreported correlations are <  0.100)
    C(sigma, amplitude)          = -0.869 



In [698]:
plt.errorbar(kalib_druck_3[0],kalib_druck_3[1],yerr=kalib_druck_3[1]*0.02)
plt.plot(np.arange(0,0.05,0.0001),out_druck_kalib_3.eval(x=np.arange(0,0.05,0.0001)))
plt.show()

### Linerare Fit

In [722]:
kalib_druck_frequency = [23.888,23.880,23.870]
kalib_druck_time = [0.04101482,0.0269,0.01103083]
kalib_druck_frequency_err = [0.000372,0.000424,0.000285]

In [723]:
mod_kalib_druck = LinearModel()

In [724]:
pars_kalib_druck = mod_kalib_druck.guess(kalib_druck_frequency, x=kalib_druck_time)

In [725]:
out_kalib_druck = mod_kalib_druck.fit(np.array(kalib_druck_frequency),
                                          params=pars_kalib_druck,
                                          x=np.array(kalib_druck_time),
                                          weights=1/np.array(kalib_druck_frequency_err))

In [726]:
print(out_kalib_druck.fit_report())

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # function evals   = 9
    # data points      = 3
    # variables        = 2
    chi-square         = 0.948
    reduced chi-square = 0.948
[[Variables]]
    slope:       0.60265693 +/- 0.015032 (2.49%) (init= 0.6009355)
    intercept:   23.8634287 +/- 0.000399 (0.00%) (init= 23.86352)
[[Correlations]] (unreported correlations are <  0.100)
    C(slope, intercept)          = -0.873 



In [727]:
plt.errorbar(kalib_druck_time,kalib_druck_frequency,yerr=np.array(kalib_druck_frequency_err),
             xerr=0.0001, fmt=' .', label="Data")
plt.plot(np.arange(0,0.05,0.0001),out_kalib_druck.eval(x=np.arange(0,0.05,0.0001)), label="Fit")
plt.gca().get_yaxis().get_major_formatter().set_useOffset(False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlabel(r'Time/s', fontsize=24)
plt.ylabel("Frequency/GHz", fontsize=24)
plt.legend(loc='best', fontsize=24)
plt.show()

## Druck abhängigkeit

In [720]:
druck_2 = np.genfromtxt("Frederik_Henrik/Messung_3/3_3_breite_2.tab", skip_header=1).T
druck_3 = np.genfromtxt("Frederik_Henrik/Messung_3/3_3_breite_3.tab", skip_header=1).T
druck_4 = np.genfromtxt("Frederik_Henrik/Messung_3/3_3_breite_4.tab", skip_header=1).T
druck_5 = np.genfromtxt("Frederik_Henrik/Messung_3/3_3_breite_5.tab", skip_header=1).T
druck_6 = np.genfromtxt("Frederik_Henrik/Messung_3/3_3_breite_6.tab", skip_header=1).T
druck_7 = np.genfromtxt("Frederik_Henrik/Messung_3/3_3_breite_7.tab", skip_header=1).T
druck_8 = np.genfromtxt("Frederik_Henrik/Messung_3/3_3_breite_8.tab", skip_header=1).T
druck_9 = np.genfromtxt("Frederik_Henrik/Messung_3/3_3_breite_9.tab", skip_header=1).T
druck_10 = np.genfromtxt("Frederik_Henrik/Messung_3/3_3_breite_10.tab", skip_header=1).T
druck_11 = np.genfromtxt("Frederik_Henrik/Messung_3/3_3_breite_11.tab", skip_header=1).T
druck_12 = np.genfromtxt("Frederik_Henrik/Messung_3/3_3_breite_12.tab", skip_header=1).T
druck_13 = np.genfromtxt("Frederik_Henrik/Messung_3/3_3_breite_13.tab", skip_header=1).T

In [721]:
plt.plot(druck_2[0],druck_2[1], label="No.1")
plt.plot(druck_3[0],druck_3[1], label="No.2")
plt.plot(druck_4[0],druck_4[1], label="No.3")
plt.plot(druck_5[0],druck_5[1], label="No.4")
plt.plot(druck_7[0],druck_7[1], label="No.5")
plt.plot(druck_7[0],druck_7[1], label="No.6")
plt.plot(druck_8[0],druck_8[1], label="No.7")
plt.plot(druck_9[0],druck_9[1], label="No.8")
plt.plot(druck_10[0],druck_10[1], label="No.9")
plt.plot(druck_11[0],druck_11[1], label="No.10")
plt.plot(druck_12[0],druck_12[1], label="No.11")
plt.plot(druck_13[0],druck_13[1], label="No.12")
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlim(0,0.05)
plt.ylim(-0.015,0.005)
plt.xlabel(r'Time/s', fontsize=24)
plt.ylabel("Frequency/GHz", fontsize=24)
plt.legend(loc='best', fontsize=20)
plt.show()

### Druck 2

#### Untergrund

In [150]:
plt.plot(druck_2[0],druck_2[1])
plt.show()

In [151]:
submask_untergrund_druck_2_1 = np.where(druck_2[0] > 0.026, True, False)
submask_untergrund_druck_2_2 = np.where(druck_2[0] < 0.017, True,False)
mask_untergrund_druck_2 = np.where(np.logical_or(submask_untergrund_druck_2_1,
                                                 submask_untergrund_druck_2_2))

In [152]:
mod_underground_druck_2 = PolynomialModel(6)

In [153]:
pars_underground_druck_2 = mod_underground_druck_2.guess(druck_2[1][mask_untergrund_druck_2],
                                                         x=druck_2[0][mask_untergrund_druck_2])

In [154]:
out_underground_druck_2 = mod_underground_druck_2.fit(druck_2[1][mask_untergrund_druck_2],
                                                      params=pars_underground_druck_2,
                                                      x=druck_2[0][mask_untergrund_druck_2],
                                                      weights=1/0.0001)

In [155]:
print(out_underground_druck_2.fit_report())

[[Model]]
    Model(polynomial)
[[Fit Statistics]]
    # function evals   = 10
    # data points      = 1685
    # variables        = 7
    chi-square         = 1985.587
    reduced chi-square = 1.183
[[Variables]]
    c0:   0.00127978 +/- 1.69e-05 (1.32%) (init= 0.001279786)
    c1:   0.59335626 +/- 0.009333 (1.57%) (init= 0.5933563)
    c2:  -54.9230219 +/- 1.606777 (2.93%) (init=-54.92302)
    c3:   1519.22208 +/- 119.4330 (7.86%) (init= 1519.222)
    c4:  -5419.51629 +/- 4.31e+03 (79.51%) (init=-5419.516)
    c5:  -3.2252e+05 +/- 7.41e+04 (22.98%) (init=-322521.7)
    c6:   3.3544e+06 +/- 4.86e+05 (14.49%) (init= 3354381)
[[Correlations]] (unreported correlations are <  0.100)
    C(c5, c6)                    = -0.996 
    C(c4, c5)                    = -0.994 
    C(c3, c4)                    = -0.989 
    C(c2, c3)                    = -0.981 
    C(c4, c6)                    =  0.980 
    C(c3, c5)                    =  0.967 
    C(c1, c2)                    = -0.965 
    C(c2,

In [156]:
out_underground_druck_2.plot()
plt.show()

In [564]:
plt.errorbar(druck_2[0],druck_2[1],yerr=0.00005,fmt=" ." ,label="Data")
plt.plot(np.arange(0,0.05,0.001),out_underground_druck_2.eval(x=np.arange(0,0.05,0.001)),color="red",label="Underground Fit")
plt.gca().get_yaxis().get_major_formatter().set_useOffset(False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlabel(r'Time/s', fontsize=24)
plt.ylabel("Current/V", fontsize=24)
#plt.xlim(0.012,0.03)
plt.legend(loc='best', fontsize=24)
plt.show()

In [158]:
plt.plot(druck_2[0],druck_2[1]-out_underground_druck_2.eval(x=druck_2[0]), ' .')
plt.show()

In [159]:
druck_untergrund = out_underground_druck_2.eval(x=druck_2[0])

In [160]:
druck_2_new = druck_2[1]-out_underground_druck_2.eval(x=druck_2[0])

#### Fit

In [161]:
mask_druck_2 = np.where(druck_2[0][np.where(druck_2[0] < 0.0222)[0]] > 0.0200)[0]

In [162]:
mod_druck_2 = LorentzianModel()

In [163]:
pars_druck_2 = mod_druck_2.guess(druck_2_new[mask_druck_2],x=druck_2[0][mask_druck_2])
pars_druck_2["amplitude"].set(-0.0001)
pars_druck_2["center"].set(0.0211)
pars_druck_2["sigma"].set(0.01)

In [164]:
out_druck_2 = mod_druck_2.fit(druck_2_new[mask_druck_2], params=pars_druck_2, x=druck_2[0][mask_druck_2], weights=1/(druck_2_new[mask_druck_2]*0.03))

In [165]:
print(out_druck_2.fit_report())

[[Model]]
    Model(lorentzian)
[[Fit Statistics]]
    # function evals   = 73
    # data points      = 87
    # variables        = 3
    chi-square         = 88.358
    reduced chi-square = 1.052
[[Variables]]
    sigma:       0.00069943 +/- 5.00e-06 (0.71%) (init= 0.01)
    fwhm:        0.00139886 +/- 1.00e-05 (0.71%)  == '2.0000000*sigma'
    center:      0.02117319 +/- 2.77e-06 (0.01%) (init= 0.0211)
    amplitude:  -7.0158e-06 +/- 2.73e-08 (0.39%) (init=-0.0001)
[[Correlations]] (unreported correlations are <  0.100)
    C(sigma, amplitude)          = -0.527 
    C(center, amplitude)         = -0.110 



In [166]:
out_druck_2.plot()
plt.show()

In [554]:
plt.errorbar(druck_2[0],druck_2_new,yerr=0.00005,fmt=" ." ,label="Data")
plt.plot(np.arange(0,0.04,0.00001),out_druck_2.eval(x=np.arange(0,0.04,0.00001)),label="Fit")
plt.gca().get_yaxis().get_major_formatter().set_useOffset(False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlabel(r'Time/s', fontsize=24)
plt.ylabel("Current/V", fontsize=24)
plt.xlim(0.012,0.03)
plt.legend(loc='best', fontsize=24)
plt.show()

### Druck 3

#### Untergrund

In [168]:
plt.plot(druck_3[0],druck_3[1]-out_underground_druck_2.eval(x=druck_2[0]))
plt.show()

In [169]:
druck_3_new = druck_3[1]-out_underground_druck_2.eval(x=druck_2[0])

#### Fit

In [170]:
mask_druck_3 = np.where(druck_3[0][np.where(druck_3[0] < 0.0225)[0]] > 0.01975)[0]

In [171]:
mod_druck_3 = LorentzianModel()
mod_const_druck_3 = ConstantModel()

In [172]:
pars_druck_3 = mod_druck_3.guess(druck_3_new[mask_druck_3],x=druck_3[0][mask_druck_3])
pars_druck_3 += mod_const_druck_3.make_params()
pars_druck_3["c"].set(-0.01)
pars_druck_3["amplitude"].set(-0.0001)
pars_druck_3["center"].set(0.0211)
pars_druck_3["sigma"].set(0.0001)

In [173]:
mod_composit_druck_3 = mod_druck_3 + mod_const_druck_3

In [174]:
out_druck_3 = mod_composit_druck_3.fit(druck_3_new[mask_druck_3], params=pars_druck_3, x=druck_3[0][mask_druck_3], weights=1/(druck_3_new[mask_druck_3]*0.03))

In [175]:
print(out_druck_3.fit_report())

[[Model]]
    (Model(lorentzian) + Model(constant))
[[Fit Statistics]]
    # function evals   = 48
    # data points      = 109
    # variables        = 4
    chi-square         = 73.782
    reduced chi-square = 0.703
[[Variables]]
    sigma:       0.00068259 +/- 8.61e-06 (1.26%) (init= 0.0001)
    fwhm:        0.00136519 +/- 1.72e-05 (1.26%)  == '2.0000000*sigma'
    center:      0.02117208 +/- 2.14e-06 (0.01%) (init= 0.0211)
    amplitude:  -9.3328e-06 +/- 1.38e-07 (1.48%) (init=-0.0001)
    c:          -0.00016069 +/- 2.50e-05 (15.57%) (init=-0.01)
[[Correlations]] (unreported correlations are <  0.100)
    C(amplitude, c)              = -0.983 
    C(sigma, amplitude)          = -0.924 
    C(sigma, c)                  =  0.921 



In [176]:
out_druck_3.plot()
plt.show()

In [177]:
plt.plot(druck_3[0],druck_3_new)
plt.plot(np.arange(0,0.04,0.00001),out_druck_3.eval(x=np.arange(0,0.04,0.00001)))
plt.show()

### Druck 4

#### Untergrund

In [178]:
plt.plot(druck_4[0],druck_4[1]-out_underground_druck_2.eval(x=druck_4[0]))
plt.show()

In [179]:
druck_4_new = druck_4[1]-out_underground_druck_2.eval(x=druck_4[0])

#### Fit

In [180]:
mask_druck_4 = np.where(druck_4[0][np.where(druck_4[0] < 0.0230)[0]] > 0.01915)[0]

In [181]:
mod_druck_4 = LorentzianModel()
mod_const_druck_4 = ConstantModel()

In [182]:
pars_druck_4 = mod_druck_4.guess(druck_4_new[mask_druck_4],x=druck_4[0][mask_druck_4])
pars_druck_4 += mod_const_druck_4.make_params()
pars_druck_4["c"].set(-0.01)
pars_druck_4["amplitude"].set(-0.0001)
pars_druck_4["center"].set(0.0211)
pars_druck_4["sigma"].set(0.01)

In [183]:
mod_composit_druck_4 = mod_druck_4 + mod_const_druck_4

In [184]:
out_druck_4 = mod_composit_druck_4.fit(druck_4_new[mask_druck_4], params=pars_druck_4, x=druck_4[0][mask_druck_4], weights=1/(druck_4_new[mask_druck_4]*0.03))

In [185]:
print(out_druck_4.fit_report())

[[Model]]
    (Model(lorentzian) + Model(constant))
[[Fit Statistics]]
    # function evals   = 133
    # data points      = 153
    # variables        = 4
    chi-square         = 143.121
    reduced chi-square = 0.961
[[Variables]]
    sigma:       0.00093206 +/- 1.19e-05 (1.27%) (init= 0.01)
    fwhm:        0.00186411 +/- 2.37e-05 (1.27%)  == '2.0000000*sigma'
    center:      0.02108456 +/- 3.02e-06 (0.01%) (init= 0.0211)
    amplitude:  -2.2007e-05 +/- 3.25e-07 (1.48%) (init=-0.0001)
    c:          -0.00042925 +/- 4.26e-05 (9.92%) (init=-0.01)
[[Correlations]] (unreported correlations are <  0.100)
    C(amplitude, c)              = -0.982 
    C(sigma, amplitude)          = -0.922 
    C(sigma, c)                  =  0.919 



In [186]:
out_druck_4.plot()
plt.show()

In [187]:
plt.plot(druck_4[0],druck_4_new)
plt.plot(np.arange(0,0.04,0.00001),out_druck_4.eval(x=np.arange(0,0.04,0.00001)))
plt.show()

### Druck 5

#### Untergrund

In [188]:
plt.plot(druck_5[0],druck_5[1]-out_underground_druck_2.eval(x=druck_5[0]))
plt.show()

In [189]:
druck_5_new = druck_5[1]-out_underground_druck_2.eval(x=druck_5[0])

#### Fit

In [190]:
mask_druck_5 = np.where(druck_5[0][np.where(druck_5[0] < 0.0248)[0]] > 0.018)[0]

In [191]:
mod_druck_5 = LorentzianModel()
mod_const_druck_5 = ConstantModel()

In [192]:
pars_druck_5 = mod_druck_5.guess(druck_5_new[mask_druck_5],x=druck_5[0][mask_druck_5])
pars_druck_5 += mod_const_druck_5.make_params()
pars_druck_5["c"].set(-0.01)
pars_druck_5["amplitude"].set(-5.9266e-05)
pars_druck_5["center"].set(0.021)
pars_druck_5["sigma"].set(0.03)

In [193]:
mod_composit_druck_5 = mod_druck_5 + mod_const_druck_5

In [194]:
out_druck_5 = mod_composit_druck_5.fit(druck_5_new[mask_druck_5], params=pars_druck_5, x=druck_5[0][mask_druck_5], weights=1/(druck_5_new[mask_druck_5]*0.03))

In [195]:
print(out_druck_5.fit_report())

[[Model]]
    (Model(lorentzian) + Model(constant))
[[Fit Statistics]]
    # function evals   = 398
    # data points      = 271
    # variables        = 4
    chi-square         = 380.724
    reduced chi-square = 1.426
[[Variables]]
    sigma:       0.00157233 +/- 1.40e-05 (0.89%) (init= 0.03)
    fwhm:        0.00314466 +/- 2.79e-05 (0.89%)  == '2.0000000*sigma'
    center:      0.02092747 +/- 4.51e-06 (0.02%) (init= 0.021)
    amplitude:  -5.5467e-05 +/- 4.98e-07 (0.90%) (init=-5.9266e-05)
    c:          -0.00020568 +/- 3.59e-05 (17.43%) (init=-0.01)
[[Correlations]] (unreported correlations are <  0.100)
    C(amplitude, c)              = -0.966 
    C(sigma, c)                  =  0.878 
    C(sigma, amplitude)          = -0.868 
    C(center, c)                 =  0.221 
    C(center, amplitude)         = -0.178 
    C(sigma, center)             =  0.108 



In [196]:
out_druck_5.plot()
plt.legend(loc='best')
plt.show()

In [197]:
plt.plot(druck_5[0],druck_5_new)
plt.plot(np.arange(0,0.04,0.00001),out_druck_5.eval(x=np.arange(0,0.04,0.00001)))
plt.show()

### Druck 6

#### Untergrund

In [198]:
plt.plot(druck_6[0],druck_6[1]-out_underground_druck_2.eval(x=druck_6[0]))
plt.show()

In [199]:
druck_6_new = druck_6[1]-out_underground_druck_2.eval(x=druck_6[0])

#### Fit

In [200]:
mask_druck_6 = np.where(druck_6[0][np.where(druck_6[0] < 0.0255)[0]] > 0.01625)[0]

In [201]:
mod_druck_6 = LorentzianModel()
mod_const_druck_6 = ConstantModel()

In [202]:
pars_druck_6 = mod_druck_6.guess(druck_6_new[mask_druck_6],x=druck_6[0][mask_druck_6])
pars_druck_6 += mod_const_druck_6.make_params()
pars_druck_6["c"].set(-0.01)
pars_druck_6["amplitude"].set(-5.9266e-05)
pars_druck_6["center"].set(0.021)
pars_druck_6["sigma"].set(0.0003)

In [203]:
mod_composit_druck_6 = mod_druck_6 + mod_const_druck_6

In [204]:
out_druck_6 = mod_composit_druck_6.fit(druck_6_new[mask_druck_6]-0.0008, params=pars_druck_6, x=druck_6[0][mask_druck_6], weights=1/(druck_6_new[mask_druck_6]*0.03))

In [205]:
print(out_druck_6.fit_report())

[[Model]]
    (Model(lorentzian) + Model(constant))
[[Fit Statistics]]
    # function evals   = 43
    # data points      = 369
    # variables        = 4
    chi-square         = 339.329
    reduced chi-square = 0.930
[[Variables]]
    sigma:       0.00280623 +/- 2.50e-05 (0.89%) (init= 0.0003)
    fwhm:        0.00561247 +/- 5.00e-05 (0.89%)  == '2.0000000*sigma'
    center:      0.02080467 +/- 4.59e-06 (0.02%) (init= 0.021)
    amplitude:  -0.00012309 +/- 1.48e-06 (1.21%) (init=-5.9266e-05)
    c:          -0.00022114 +/- 7.24e-05 (32.76%) (init=-0.01)
[[Correlations]] (unreported correlations are <  0.100)
    C(amplitude, c)              = -0.992 
    C(sigma, amplitude)          = -0.955 
    C(sigma, c)                  =  0.946 



In [206]:
out_druck_6.plot()
plt.show()

In [207]:
plt.plot(druck_6[0],druck_6_new-0.0008)
plt.plot(np.arange(0,0.04,0.00001),out_druck_6.eval(x=np.arange(0,0.04,0.00001)))
plt.show()

### Druck 7

#### Untergrund

In [208]:
plt.plot(druck_7[0],druck_7[1]-out_underground_druck_2.eval(x=druck_7[0]))
plt.show()

In [209]:
druck_7_new = druck_7[1]-out_underground_druck_2.eval(x=druck_7[0])

#### Fit

In [210]:
mask_druck_7 = np.where(druck_7[0][np.where(druck_7[0] < 0.027)[0]] > 0.015)[0]

In [211]:
mod_druck_7 = LorentzianModel()
mod_const_druck_7 = ConstantModel()

In [212]:
mod_composit_druck_7 = mod_druck_7 + mod_const_druck_7

In [213]:
pars_druck_7 = mod_druck_7.guess(druck_7_new[mask_druck_7],x=druck_7[0][mask_druck_7])
pars_druck_7 += mod_const_druck_7.make_params()
pars_druck_7["c"].set(-0.01)
pars_druck_7["amplitude"].set(-5.9266e-05)
pars_druck_7["center"].set(0.021)
pars_druck_7["sigma"].set(0.000003)

In [214]:
out_druck_7 = mod_composit_druck_7.fit(druck_7_new[mask_druck_7]-0.0011, params=pars_druck_7, x=druck_7[0][mask_druck_7], weights=1/(druck_7_new[mask_druck_7]*0.03))

In [215]:
print(out_druck_7.fit_report())

[[Model]]
    (Model(lorentzian) + Model(constant))
[[Fit Statistics]]
    # function evals   = 78
    # data points      = 479
    # variables        = 4
    chi-square         = 368.438
    reduced chi-square = 0.776
[[Variables]]
    sigma:       0.00412140 +/- 3.15e-05 (0.76%) (init= 3e-06)
    fwhm:        0.00824281 +/- 6.29e-05 (0.76%)  == '2.0000000*sigma'
    center:      0.02095562 +/- 4.62e-06 (0.02%) (init= 0.021)
    amplitude:  -0.00019992 +/- 2.26e-06 (1.13%) (init=-5.9266e-05)
    c:           0.00067156 +/- 7.96e-05 (11.85%) (init=-0.01)
[[Correlations]] (unreported correlations are <  0.100)
    C(amplitude, c)              = -0.995 
    C(sigma, amplitude)          = -0.969 
    C(sigma, c)                  =  0.960 



In [216]:
out_druck_7.plot()
plt.show()

In [217]:
plt.plot(druck_7[0],druck_7_new-0.0011)
plt.plot(np.arange(0,0.04,0.00001),out_druck_7.eval(x=np.arange(0,0.04,0.00001)))
plt.show()

### Druck 8

#### Untergrund

In [218]:
plt.plot(druck_8[0],druck_8[1]-out_underground_druck_2.eval(x=druck_8[0]))
plt.show()

In [219]:
druck_8_new = druck_8[1]-out_underground_druck_2.eval(x=druck_8[0])

#### Fit

In [220]:
mask_druck_8 = np.where(druck_8[0][np.where(druck_8[0] < 0.04)[0]] > 0.015)[0]

In [221]:
mod_druck_8 = LorentzianModel()
mod_const_druck_8 = ConstantModel()

In [222]:
mod_composit_druck_8 = mod_druck_8 + mod_const_druck_8

In [223]:
pars_druck_8 = mod_druck_8.guess(druck_8_new[mask_druck_8],x=druck_8[0][mask_druck_8])
pars_druck_8 += mod_const_druck_8.make_params()
pars_druck_8["c"].set(-0.01)
pars_druck_8["amplitude"].set(-5.9266e-05)
pars_druck_8["center"].set(0.021, vary=True)
pars_druck_8["sigma"].set(0.000003)

In [224]:
out_druck_8 = mod_composit_druck_8.fit(druck_8_new[mask_druck_8]-0.0011, params=pars_druck_8, x=druck_8[0][mask_druck_8], weights=1/(druck_8_new[mask_druck_8]*0.05))

In [225]:
print(out_druck_8.fit_report())

[[Model]]
    (Model(lorentzian) + Model(constant))
[[Fit Statistics]]
    # function evals   = 85
    # data points      = 999
    # variables        = 4
    chi-square         = 7461.709
    reduced chi-square = 7.499
[[Variables]]
    sigma:       0.00649954 +/- 5.71e-05 (0.88%) (init= 3e-06)
    fwhm:        0.01299909 +/- 0.000114 (0.88%)  == '2.0000000*sigma'
    center:      0.02121198 +/- 3.10e-05 (0.15%) (init= 0.021)
    amplitude:  -0.00036587 +/- 2.68e-06 (0.73%) (init=-5.9266e-05)
    c:           0.00326261 +/- 3.98e-05 (1.22%) (init=-0.01)
[[Correlations]] (unreported correlations are <  0.100)
    C(amplitude, c)              = -0.910 
    C(sigma, c)                  =  0.850 
    C(sigma, amplitude)          = -0.780 
    C(center, amplitude)         =  0.603 
    C(sigma, center)             = -0.583 
    C(center, c)                 = -0.321 



In [226]:
out_druck_8.plot()
plt.show()

In [227]:
plt.plot(druck_8[0],druck_8_new-0.0011)
plt.plot(np.arange(0,0.04,0.00001),out_druck_8.eval(x=np.arange(0,0.04,0.00001)))
plt.show()

### Druck 9

#### Untergrund

In [228]:
plt.plot(druck_9[0],druck_9[1]-out_underground_druck_2.eval(x=druck_9[0]))
plt.show()

In [229]:
druck_9_new = druck_9[1]-out_underground_druck_2.eval(x=druck_9[0])

#### Fit

In [230]:
mask_druck_9 = np.where(druck_9[0][np.where(druck_9[0] < 0.046)[0]] > 0.003)[0]

In [231]:
mod_druck_9 = LorentzianModel()
mod_const_druck_9 = ConstantModel()

In [232]:
pars_druck_9 = mod_druck_9.guess(druck_9_new[mask_druck_9],x=druck_9[0][mask_druck_9])
pars_druck_9 += mod_const_druck_9.make_params()
pars_druck_9["c"].set(-0.01)
pars_druck_9["amplitude"].set(-5.9266e-05)
pars_druck_9["center"].set(0.021)
pars_druck_9["sigma"].set(0.007,vary=True)

In [233]:
mod_composit_druck_9 = mod_druck_9 + mod_const_druck_9

In [234]:
out_druck_9 = mod_composit_druck_9.fit(druck_9_new[mask_druck_9], params=pars_druck_9, x=druck_9[0][mask_druck_9], weights=1/(druck_9_new[mask_druck_9]*0.02))

In [235]:
print(out_druck_9.fit_report())

[[Model]]
    (Model(lorentzian) + Model(constant))
[[Fit Statistics]]
    # function evals   = 84
    # data points      = 1719
    # variables        = 4
    chi-square         = 526834.615
    reduced chi-square = 307.192
[[Variables]]
    sigma:       0.00828770 +/- 0.000127 (1.53%) (init= 0.007)
    fwhm:        0.01657540 +/- 0.000253 (1.53%)  == '2.0000000*sigma'
    center:      0.02074566 +/- 2.64e-06 (0.01%) (init= 0.021)
    amplitude:  -0.00042180 +/- 4.88e-06 (1.16%) (init=-5.9266e-05)
    c:           0.00392686 +/- 6.91e-05 (1.76%) (init=-0.01)
[[Correlations]] (unreported correlations are <  0.100)
    C(amplitude, c)              = -0.937 
    C(sigma, c)                  =  0.857 
    C(sigma, amplitude)          = -0.622 
    C(center, amplitude)         =  0.121 
    C(center, c)                 = -0.116 



In [236]:
out_druck_9.plot()
plt.show()

In [237]:
plt.plot(druck_9[0],druck_9_new)
plt.plot(np.arange(0,0.04,0.00001),out_druck_9.eval(x=np.arange(0,0.04,0.00001)))
plt.show()

### Druck 10

#### Untergrund

In [238]:
plt.plot(druck_10[0],druck_10[1]-out_underground_druck_2.eval(x=druck_10[0]))
plt.show()

In [239]:
druck_10_new = druck_10[1]-out_underground_druck_2.eval(x=druck_10[0])

### Fit

In [240]:
mask_druck_10 = np.where(druck_10[0][np.where(druck_10[0] < 0.032)[0]] > 0.009)[0]

In [241]:
mod_druck_10 = LorentzianModel()
mod_const_druck_10 = ConstantModel()

In [242]:
pars_druck_10 = mod_druck_10.guess(druck_10_new[mask_druck_10],x=druck_10[0][mask_druck_10])
pars_druck_10 += mod_const_druck_10.make_params()
pars_druck_10["c"].set(-0.01)
pars_druck_10["amplitude"].set(-5.9266e-05)
pars_druck_10["center"].set(0.021)
pars_druck_10["sigma"].set(0.0001)

In [243]:
mod_composit_druck_10 = mod_druck_10 + mod_const_druck_10

In [244]:
out_druck_10 = mod_composit_druck_10.fit(druck_10_new[mask_druck_10]-0.002, params=pars_druck_10, x=druck_10[0][mask_druck_10], weights=1/(druck_10_new[mask_druck_10]*0.02))

In [245]:
print(out_druck_10.fit_report())

[[Model]]
    (Model(lorentzian) + Model(constant))
[[Fit Statistics]]
    # function evals   = 58
    # data points      = 919
    # variables        = 4
    chi-square         = 1087.745
    reduced chi-square = 1.189
[[Variables]]
    sigma:       0.00997449 +/- 5.07e-05 (0.51%) (init= 0.0001)
    fwhm:        0.01994898 +/- 0.000101 (0.51%)  == '2.0000000*sigma'
    center:      0.02077592 +/- 4.73e-06 (0.02%) (init= 0.021)
    amplitude:  -0.00047786 +/- 4.20e-06 (0.88%) (init=-5.9266e-05)
    c:           0.00221470 +/- 6.70e-05 (3.02%) (init=-0.01)
[[Correlations]] (unreported correlations are <  0.100)
    C(amplitude, c)              = -0.998 
    C(sigma, amplitude)          = -0.984 
    C(sigma, c)                  =  0.977 
    C(center, c)                 = -0.114 
    C(center, amplitude)         =  0.109 



In [246]:
out_druck_10.plot()
plt.show()

In [247]:
plt.plot(druck_10[0],druck_10_new-0.002)
plt.plot(np.arange(0,0.04,0.00001),out_druck_10.eval(x=np.arange(0,0.04,0.00001)))
plt.show()

### Druck 11

#### Untergrund

In [248]:
plt.plot(druck_11[0],druck_11[1])
plt.show()

In [249]:
druck_11_new = druck_11[1]

#### Fit

In [250]:
mask_druck_11 = np.where(druck_11[0][np.where(druck_11[0] < 0.033)[0]] > 0.0114)[0]

In [251]:
mod_druck_11 = LorentzianModel()
mod_const_druck_11 = ConstantModel()

In [252]:
pars_druck_11 = mod_druck_11.guess(druck_11_new[mask_druck_11],x=druck_11[0][mask_druck_11])
pars_druck_11 += mod_const_druck_11.make_params()
pars_druck_11["c"].set(-0.01)
pars_druck_11["amplitude"].set(-5.9266e-05)
pars_druck_11["center"].set(0.021)
pars_druck_11["sigma"].set(0.00001)

In [253]:
mod_composit_druck_11 = mod_druck_11 + mod_const_druck_11

In [254]:
out_druck_11 = mod_composit_druck_11.fit(druck_11_new[mask_druck_11], params=pars_druck_11, x=druck_11[0][mask_druck_11], weights=(1/((druck_11_new[mask_druck_11]++0.00001)*0.02)))

In [255]:
print(out_druck_11.fit_report())

[[Model]]
    (Model(lorentzian) + Model(constant))
[[Fit Statistics]]
    # function evals   = 74
    # data points      = 863
    # variables        = 4
    chi-square         = 2982.741
    reduced chi-square = 3.472
[[Variables]]
    sigma:       0.01124635 +/- 0.000113 (1.00%) (init= 1e-05)
    fwhm:        0.02249271 +/- 0.000225 (1.00%)  == '2.0000000*sigma'
    center:      0.02200069 +/- 6.75e-06 (0.03%) (init= 0.021)
    amplitude:  -0.00051044 +/- 9.88e-06 (1.94%) (init=-5.9266e-05)
    c:           0.00594950 +/- 0.000149 (2.50%) (init=-0.01)
[[Correlations]] (unreported correlations are <  0.100)
    C(amplitude, c)              = -0.999 
    C(sigma, amplitude)          = -0.991 
    C(sigma, c)                  =  0.986 



In [256]:
out_druck_11.plot()
plt.show()

In [257]:
plt.plot(druck_11[0],druck_11_new)
plt.plot(np.arange(0,0.04,0.00001),out_druck_11.eval(x=np.arange(0,0.04,0.00001)))
plt.show()

### Druck 12

#### Untergrund

In [258]:
plt.plot(druck_12[0],druck_12[1])
plt.show()

In [259]:
druck_12_new = druck_12[1]

#### Fit

In [260]:
mask_druck_12 = np.where(druck_12[0][np.where(druck_12[0] < 0.033)[0]] > 0.0114)[0]

In [261]:
mod_druck_12 = LorentzianModel()
mod_const_druck_12 = ConstantModel()

In [262]:
pars_druck_12 = mod_druck_12.guess(druck_12_new[mask_druck_12],x=druck_12[0][mask_druck_12])
pars_druck_12 += mod_const_druck_12.make_params()
pars_druck_12["c"].set(-0.01)
pars_druck_12["amplitude"].set(-5.9266e-05)
pars_druck_12["center"].set(0.021)
pars_druck_12["sigma"].set(0.0001)

In [263]:
mod_composit_druck_12 = mod_druck_12 + mod_const_druck_12

In [264]:
out_druck_12 = mod_composit_druck_12.fit(druck_12_new[mask_druck_12], params=pars_druck_12, x=druck_12[0][mask_druck_12], weights=(1/((druck_12_new[mask_druck_11]++0.00001)*0.02)))

In [265]:
print(out_druck_12.fit_report())

[[Model]]
    (Model(lorentzian) + Model(constant))
[[Fit Statistics]]
    # function evals   = 81
    # data points      = 863
    # variables        = 4
    chi-square         = 1997.858
    reduced chi-square = 2.326
[[Variables]]
    sigma:       0.01245460 +/- 0.000137 (1.10%) (init= 0.0001)
    fwhm:        0.02490921 +/- 0.000274 (1.10%)  == '2.0000000*sigma'
    center:      0.02195080 +/- 6.87e-06 (0.03%) (init= 0.021)
    amplitude:  -0.00049499 +/- 1.13e-05 (2.28%) (init=-5.9266e-05)
    c:           0.00540525 +/- 0.000159 (2.95%) (init=-0.01)
[[Correlations]] (unreported correlations are <  0.100)
    C(amplitude, c)              = -0.999 
    C(sigma, amplitude)          = -0.994 
    C(sigma, c)                  =  0.990 
    C(center, c)                 =  0.105 
    C(center, amplitude)         = -0.101 



In [266]:
out_druck_12.plot()
plt.show()

In [267]:
plt.plot(druck_12[0],druck_12_new)
plt.plot(np.arange(0,0.04,0.00001),out_druck_12.eval(x=np.arange(0,0.04,0.00001)))
plt.show()

### Druck 13

#### Untergrund

In [268]:
plt.plot(druck_13[0],druck_13[1])
plt.show()

In [269]:
druck_13_new = druck_13[1]

#### Fit

In [270]:
mask_druck_13 = np.where(druck_13[0][np.where(druck_13[0] < 0.025)[0]] > 0.015)[0]

In [271]:
mod_druck_13 = LorentzianModel()
mod_const_druck_13 = ConstantModel()

In [272]:
pars_druck_13 = mod_druck_13.guess(druck_13_new[mask_druck_13],x=druck_13[0][mask_druck_13])
pars_druck_13 += mod_const_druck_13.make_params()
pars_druck_13["c"].set(-0.01)
pars_druck_13["amplitude"].set(-5.9266e-05)
pars_druck_13["center"].set(0.021)
pars_druck_13["sigma"].set(0.00001)

In [273]:
mod_composit_druck_13 = mod_druck_13 + mod_const_druck_13

In [274]:
out_druck_13 = mod_composit_druck_13.fit(druck_13_new[mask_druck_13], params=pars_druck_13, x=druck_13[0][mask_druck_13], weights=(1/((druck_13_new[mask_druck_13]+0.00001)*0.02)))

In [275]:
print(out_druck_13.fit_report())

[[Model]]
    (Model(lorentzian) + Model(constant))
[[Fit Statistics]]
    # function evals   = 312
    # data points      = 399
    # variables        = 4
    chi-square         = 219.517
    reduced chi-square = 0.556
[[Variables]]
    sigma:       0.01643498 +/- 0.001401 (8.52%) (init= 1e-05)
    fwhm:        0.03286996 +/- 0.002802 (8.52%)  == '2.0000000*sigma'
    center:      0.02183337 +/- 2.54e-05 (0.12%) (init= 0.021)
    amplitude:  -0.00073734 +/- 0.000171 (23.15%) (init=-5.9266e-05)
    c:           0.00838228 +/- 0.002094 (24.99%) (init=-0.01)
[[Correlations]] (unreported correlations are <  0.100)
    C(amplitude, c)              = -1.000 
    C(sigma, amplitude)          = -0.999 
    C(sigma, c)                  =  0.998 
    C(sigma, center)             =  0.272 
    C(center, amplitude)         = -0.239 
    C(center, c)                 =  0.221 



In [276]:
out_druck_13.plot()
plt.show()

In [277]:
plt.plot(druck_13[0],druck_13_new)
plt.plot(np.arange(0,0.04,0.00001),out_druck_13.eval(x=np.arange(0,0.04,0.00001)))
plt.show()

## Breite zu Druck

In [768]:
breite = []
breite.append(out_druck_2.best_values["sigma"]*2)
breite.append(out_druck_3.best_values["sigma"]*2)
breite.append(out_druck_4.best_values["sigma"]*2)
breite.append(out_druck_5.best_values["sigma"]*2)
breite.append(out_druck_6.best_values["sigma"]*2)
breite.append(out_druck_7.best_values["sigma"]*2)
breite.append(out_druck_8.best_values["sigma"]*2)
breite.append(out_druck_9.best_values["sigma"]*2)
breite.append(out_druck_10.best_values["sigma"]*2)
breite.append(out_druck_11.best_values["sigma"]*2)
breite.append(out_druck_12.best_values["sigma"]*2)
breite.append(out_druck_13.best_values["sigma"]*2)
breite = np.array(breite)*2
breite_hz = out_kalib_druck.best_values["slope"]*breite * 1000

In [810]:
breite_err = np.array([0.00005,7.10e-05,6.72e-05,8.37e-05,7.79e-05,9.00e-05,0.000114,0.000253,0.000101,0.000225,0.000274,0.0004802])*10
breite_hz_err = out_kalib_druck.best_values["slope"]*breite_err * 1000

In [811]:
breite

array([ 0.00279772,  0.00273039,  0.00372823,  0.00628932,  0.01122495,
        0.01648564,  0.02599819,  0.03315081,  0.03989798,  0.04498543,
        0.04981843,  0.06573993])

In [812]:
breite_err

array([ 0.0005  ,  0.00071 ,  0.000672,  0.000837,  0.000779,  0.0009  ,
        0.00114 ,  0.00253 ,  0.00101 ,  0.00225 ,  0.00274 ,  0.004802])

In [813]:
druck = np.array([1.0*10**-2,
                  2.6*10**-2,
                  8.7*10**-2,
                  1.5*10**-1,
                  2.7*10**-1,
                  3.8*10**-1,
                  5.8*10**-1,
                  9.3*10**-1,
                  1.1,
                  1.3,
                  1.6,
                  2.0])

In [814]:
plt.plot(druck,breite_hz, '. ')
plt.gca().get_yaxis().get_major_formatter().set_useOffset(False)
plt.show()

In [815]:
mod_druck_breite = LinearModel()

In [816]:
pars_druck_breite = mod_druck_breite.guess(breite_hz,x=druck)

In [820]:
out_druck_breite = mod_druck_breite.fit(breite_hz,x=druck,params=pars_druck_breite,weights=1/(breite_hz_err))

In [821]:
print(out_druck_breite.fit_report())

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # function evals   = 8
    # data points      = 12
    # variables        = 2
    chi-square         = 29.113
    reduced chi-square = 2.911
[[Variables]]
    slope:       20.5358126 +/- 0.749564 (3.65%) (init= 19.02412)
    intercept:   1.27800114 +/- 0.324843 (25.42%) (init= 1.840204)
[[Correlations]] (unreported correlations are <  0.100)
    C(slope, intercept)          = -0.568 



In [836]:
plt.plot(np.arange(-1,4,0.01),out_druck_breite.eval(x=np.arange(-1,4,0.01)),label="Fit")
plt.errorbar(druck,breite_hz,yerr=breite_hz_err,xerr=0.1, fmt=' .',label="Data")
plt.gca().get_yaxis().get_major_formatter().set_useOffset(False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlim(-0.1,2.2)
plt.ylim(-0.05,45)
plt.xlabel(r'Pressure/mbar', fontsize=24)
plt.ylabel("Line width/MHz", fontsize=24)
plt.legend(loc='best', fontsize=20)
plt.show()

In [287]:
breite

array([ 0.00139886,  0.0013652 ,  0.00186412,  0.00314466,  0.00561247,
        0.00824282,  0.0129991 ,  0.0165754 ,  0.01994899,  0.02249271,
        0.02490921,  0.03286996])

In [784]:
((300/273)**0.5*20)

20.965696734438367

In [785]:
1*(300/273)**0.5

1.0482848367219184

In [786]:
((300/273)**0.5*20)*(1/0.75)

27.95426231258449

In [788]:
1*(300/273)**0.5*(1/0.75)

1.3977131156292244

In [912]:
1-28/29

0.03448275862068961